In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import tensorflow as tf
import numpy as np


#  Import and read the data.csv.
import pandas as pd 
data_df = pd.read_csv("archive/data.csv")
#data_df.head()

In [2]:
#Drop unnecessary columns
data_df = data_df.drop(["Unnamed: 0","Year","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","City", "Nat_Pop", "GDP"], 1)

#data_df = data_df.drop(["GDP_Per_Cap"], 1)
data_df = data_df[data_df.GDP_Per_Cap != '#DIV/0!']

#Rename Global_Pop% and Rural_Pop% to make it easier to call in future
data_df.rename(columns = {'Global_Pop%':'Global_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Rural_Pop%':'Rural_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Global_GDP%':'Global_GDP_Perc'}, inplace = True)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3502521125.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Unnamed: 0","Year","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","City", "Nat_Pop", "GDP"], 1)


In [3]:
#Filter out season data to just summer
data_df = data_df[data_df["Season"].str.contains("Winter") == False]

In [4]:
#Turn Medal Values into Boolean
data_df.fillna(value = False,
          inplace = True)

#Drop Season since no longer needed after filtering
data_df = data_df.drop(["Season"],1)
data_df.head(10)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1206795605.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Season"],1)


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,NOR,Handball,Gold,0,0.000705,21.474,80.592683,97101.38727,0.007291
51,ROU,Weightlifting,False,0,0.002648,45.251,75.012195,9540.579969,0.00248
52,NOR,Wrestling,False,0,0.000734,23.919,78.634146,38076.43209,0.005089
53,NOR,Wrestling,False,0,0.000713,22.725,79.841463,57492.41601,0.006027
58,NLD,Water Polo,False,0,0.003030,33.681,76.233171,9844.559262,0.011736
59,NLD,Hockey,Bronze,0,0.002891,32.094,76.890244,17547.31372,0.01356


In [5]:
#Replace medal values to true to finish boolean
data_df = data_df.replace({"Gold" : True})
data_df = data_df.replace({"Silver" : True})
data_df = data_df.replace({"Bronze" : True})

In [6]:
#Begin binning data to make it easier to get dummies
#Find min and max of data types to create bins
data_df['Global_Pop_Perc'] = data_df['Global_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Global_Pop_Perc'].min()
max_value = data_df['Global_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]
print(bins)
print (names)


1.45e-06
0.224549844
[1.45000000e-06 2.49512716e-02 4.99010931e-02 7.48509147e-02
 9.98007362e-02 1.24750558e-01 1.49700379e-01 1.74650201e-01
 1.99600022e-01 2.24549844e-01]
['0', '1', '2', '3', '4', '5', '6', '7', '8']


In [7]:
data_df["Global_Pop_Bin"] = pd.cut(data_df["Global_Pop_Perc"], bins, labels=names)
data_df

,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811,8
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286,7
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
50,NOR,Handball,True,0,0.000705,21.474,80.592683,97101.38727,0.007291,0
...,...,...,...,...,...,...,...,...,...,...
204023,ARG,Hockey,False,0,0.006017,10.101,74.603,4260.406805,0.003767,0
204024,USA,Football,False,0,0.054676,26.377,71.156098,6098.258185,0.339523,2
204025,RUS,Rowing,False,0,0.023961,26.65,65.483659,1773.575078,0.007738,0
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,4102.25644,0.013493,0


In [8]:
data_df['GDP_Per_Cap'] = data_df['GDP_Per_Cap'].astype(float, errors = 'raise')

min_value = data_df['GDP_Per_Cap'].min()
max_value = data_df['GDP_Per_Cap'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
192989.1895


In [9]:
data_df["GDP_Per_Cap"] = pd.cut(data_df["GDP_Per_Cap"], bins, labels=names)

In [10]:
data_df['Rural_Pop_Perc'] = data_df['Rural_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Rural_Pop_Perc'].min()
max_value = data_df['Rural_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
96.305


In [11]:
data_df["Rural_Pop_Bin"] = pd.cut(data_df["Rural_Pop_Perc"], bins, labels=names)

In [12]:
data_df['Life_Expect'] = data_df['Life_Expect'].astype(float, errors = 'raise')

min_value = data_df['Life_Expect'].min()
max_value = data_df['Life_Expect'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

data_df["Life_Expect_Bin"] = pd.cut(data_df["Life_Expect"], bins, labels=names)

0.0
85.41707317


In [13]:
data_df.dtypes

NOC                  object
Sport                object
Medal                  bool
Host                  int64
Global_Pop_Perc     float64
Rural_Pop_Perc      float64
Life_Expect         float64
GDP_Per_Cap        category
Global_GDP_Perc      object
Global_Pop_Bin     category
Rural_Pop_Bin      category
Life_Expect_Bin    category
dtype: object

In [14]:
min_value = data_df['Global_GDP_Perc'].min()
max_value = data_df['Global_GDP_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]

data_df["Global_GDP_Perc_Bin"] = pd.cut(data_df["Global_GDP_Perc"], bins, labels=names)

False
0.3963503649635037


In [15]:
data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)
data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4282354263.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)


,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0,8,6,7,0
1,CHN,Judo,False,0,0,7,4,7,2
26,FIN,Badminton,False,0,1,0,1,8,0
29,FIN,Athletics,False,0,1,0,1,8,0
50,NOR,Handball,True,0,4,0,2,8,0


Athletics Logistic Regression

In [16]:
# Creating on the basis of Athlethics
Athletics_df = data_df[data_df['Sport'] == "Athletics"]
Athletics_df = Athletics_df.dropna()
 
print(Athletics_df)

        NOC      Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
29      FIN  Athletics  False     0           1              0             1   
62      FRA  Athletics  False     0           1              0             1   
81      EST  Athletics  False     0           0              0             2   
82      EST  Athletics  False     0           0              0             2   
91      FRA  Athletics  False     0           0              0             2   
...     ...        ...    ...   ...         ...            ...           ...   
203977  KAZ  Athletics  False     0           0              0             4   
203978  KAZ  Athletics  False     0           0              0             4   
203979  KAZ  Athletics  False     0           0              0             4   
204019  RUS  Athletics   True     0           0              0             2   
204020  RUS  Athletics   True     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_

In [17]:
Athletics_dummies = pd.get_dummies(Athletics_df)
Athletics_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
29,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
62,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
81,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split

# Convert categorical data to numeric and separate target feature for training data
x_train = Athletics_dummies.drop(columns=['Medal'])
y_train = Athletics_dummies['Medal']

# Convert categorical data to numeric and separate target feature for testing data
x_test = Athletics_dummies.drop(columns=['Medal'])
y_test = Athletics_dummies['Medal']

In [19]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9218859667230915

In [20]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Athletics_df)

array([0.45341164, 0.44135044, 0.3849714 , ..., 0.26818746, 0.73570708,
       0.73570708])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
29,FIN,Athletics,False,0,1,0,1,8,0
62,FRA,Athletics,False,0,1,0,1,8,0
81,EST,Athletics,False,0,0,0,2,7,0
82,EST,Athletics,False,0,0,0,2,7,0
91,FRA,Athletics,False,0,0,0,2,7,1
...,...,...,...,...,...,...,...,...,...
203977,KAZ,Athletics,False,0,0,0,4,7,0
203978,KAZ,Athletics,False,0,0,0,4,7,0
203979,KAZ,Athletics,False,0,0,0,4,7,0
204019,RUS,Athletics,True,0,0,0,2,6,0


In [21]:
Athletics_df['Predictions'] = y_pred.tolist()

In [22]:
Athletics_final = Athletics_df.drop_duplicates(subset=['NOC'], keep='last')
Athletics_final = Athletics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Athletics_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3032431419.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Athletics_final = Athletics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
3132,OMN,Athletics,0.056705
93515,PLW,Athletics,0.142449
124976,SOM,Athletics,0.039488
130609,AGO,Athletics,0.039488
130654,BDI,Athletics,0.394084
...,...,...,...
203957,DEU,Athletics,0.464096
203959,NLD,Athletics,0.186879
203968,USA,Athletics,0.858038
203979,KAZ,Athletics,0.268187


In [23]:
# Creating on the basis of Athlethics
Hockey_df = data_df[data_df['Sport'] == "Hockey"]
Hockey_df = Hockey_df.dropna()
 
print(Hockey_df)

        NOC   Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
59      NLD  Hockey   True     0           0              0             2   
177     PAK  Hockey  False     0           0              0             6   
178     PAK  Hockey  False     0           0              0             6   
179     PAK  Hockey  False     0           0              1             5   
180     MYS  Hockey  False     0           0              0             5   
...     ...     ...    ...   ...         ...            ...           ...   
203941  DEU  Hockey   True     0           1              0             2   
203950  DEU  Hockey   True     0           1              0             2   
203958  NLD  Hockey  False     0           0              0             3   
204022  ARG  Hockey  False     0           0              0             1   
204023  ARG  Hockey  False     0           0              0             0   

       Life_Expect_Bin Global_GDP_Perc_Bin  
59                   8        

In [24]:
Hockey_dummies = pd.get_dummies(Hockey_df)
Hockey_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
59,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
177,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
178,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [25]:
from sklearn.model_selection import train_test_split

# Convert categorical data to numeric and separate target feature for training data
x_train = Hockey_dummies.drop(columns=['Medal'])
y_train = Hockey_dummies['Medal']

# Convert categorical data to numeric and separate target feature for testing data
x_test = Hockey_dummies.drop(columns=['Medal'])
y_test = Hockey_dummies['Medal']

In [26]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8184682501211827

In [27]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Hockey_df)

array([0.94596698, 0.59112687, 0.59112687, ..., 0.3232276 , 0.4475204 ,
       0.65699689])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
59,NLD,Hockey,True,0,0,0,2,8,0
177,PAK,Hockey,False,0,0,0,6,6,0
178,PAK,Hockey,False,0,0,0,6,6,0
179,PAK,Hockey,False,0,0,1,5,6,0
180,MYS,Hockey,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203941,DEU,Hockey,True,0,1,0,2,8,1
203950,DEU,Hockey,True,0,1,0,2,8,1
203958,NLD,Hockey,False,0,0,0,3,7,0
204022,ARG,Hockey,False,0,0,0,1,7,0


In [28]:
Hockey_df['Predictions'] = y_pred.tolist()

In [29]:
Hockey_final = Hockey_df.drop_duplicates(subset=['NOC'], keep='last')
Hockey_final = Hockey_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Hockey_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1908904570.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Hockey_final = Hockey_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
138866,DNK,Hockey,0.123520
166523,HKG,Hockey,0.087214
166601,UGA,Hockey,0.003402
172662,AUT,Hockey,0.005397
178278,EGY,Hockey,0.000385
186867,BRA,Hockey,0.092658
188387,KEN,Hockey,0.000262
188411,CUB,Hockey,0.092640
190055,MEX,Hockey,0.000858
193433,ZWE,Hockey,0.936410


In [30]:
Hockey_final.rename(columns = {'Predictions':'Hockey'}, inplace = True)
Hockey_final = Hockey_final.drop(["Sport"], 1)
Hockey_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4121295266.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Hockey_final = Hockey_final.drop(["Sport"], 1)


,NOC,Hockey
138866,DNK,0.123520
166523,HKG,0.087214
166601,UGA,0.003402
172662,AUT,0.005397
178278,EGY,0.000385
186867,BRA,0.092658
188387,KEN,0.000262
188411,CUB,0.092640
190055,MEX,0.000858
193433,ZWE,0.936410


##Swimming Logistics Regression

In [31]:
# Creating on the basis of Athlethics
Swimming_df = data_df[data_df['Sport'] == "Swimming"]
Swimming_df = Swimming_df.dropna()
  
print(Swimming_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
64      NLD  Swimming  False     0           1              0             2   
65      NLD  Swimming  False     0           1              0             2   
66      NLD  Swimming  False     0           1              0             2   
67      NLD  Swimming  False     0           1              0             2   
68      NLD  Swimming  False     0           1              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203887  RUS  Swimming  False     0           0              0             2   
203888  RUS  Swimming  False     0           0              0             2   
203942  NLD  Swimming  False     0           1              0             2   
203943  NLD  Swimming  False     0           1              0             2   
203944  NLD  Swimming   True     0           1              0             1   

       Life_Expect_Bin Global_GDP_Perc_Bin  
64    

In [32]:
Swimming_dummies = pd.get_dummies(Swimming_df)
Swimming_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
64,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
65,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
66,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [33]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Swimming_dummies.drop(columns=['Medal'])
y_train = Swimming_dummies['Medal']

In [34]:
x_test = Swimming_dummies.drop(columns=['Medal'])
y_test = Swimming_dummies['Medal']

In [35]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9132554711418528

In [36]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Swimming_df)

array([0.57587672, 0.57587672, 0.57587672, ..., 0.57587672, 0.57587672,
       0.6426502 ])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
64,NLD,Swimming,False,0,1,0,2,8,0
65,NLD,Swimming,False,0,1,0,2,8,0
66,NLD,Swimming,False,0,1,0,2,8,0
67,NLD,Swimming,False,0,1,0,2,8,0
68,NLD,Swimming,False,0,1,0,2,8,0
...,...,...,...,...,...,...,...,...,...
203887,RUS,Swimming,False,0,0,0,2,7,0
203888,RUS,Swimming,False,0,0,0,2,7,0
203942,NLD,Swimming,False,0,1,0,2,8,0
203943,NLD,Swimming,False,0,1,0,2,8,0


In [37]:
Swimming_df['Predictions'] = y_pred.tolist()

In [38]:
Swimming_final = Swimming_df.drop_duplicates(subset=['NOC'], keep='last')
Swimming_final = Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2203429247.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Swimming_final = Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [39]:
A = pd.merge(Athletics_final, Swimming_final, how='outer', on=['NOC', 'NOC'])
A

,NOC,Sport_x,Predictions_x,Sport_y,Predictions_y
0,OMN,Athletics,0.056705,Swimming,0.035615
1,PLW,Athletics,0.142449,Swimming,0.035615
2,SOM,Athletics,0.039488,NaN,NaN
3,AGO,Athletics,0.039488,Swimming,0.006518
4,BDI,Athletics,0.394084,Swimming,0.007110
...,...,...,...,...,...
182,DEU,Athletics,0.464096,Swimming,0.635457
183,NLD,Athletics,0.186879,Swimming,0.642650
184,USA,Athletics,0.858038,Swimming,0.964735
185,KAZ,Athletics,0.268187,Swimming,0.073423


In [40]:
A.rename(columns = {'Predictions_x':'Athletics', 'Predictions_y':'Swimming'}, inplace = True)
A = A.drop(["Sport_x","Sport_y"], 1)
A

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2218725990.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  A = A.drop(["Sport_x","Sport_y"], 1)


,NOC,Athletics,Swimming
0,OMN,0.056705,0.035615
1,PLW,0.142449,0.035615
2,SOM,0.039488,NaN
3,AGO,0.039488,0.006518
4,BDI,0.394084,0.007110
...,...,...,...
182,DEU,0.464096,0.635457
183,NLD,0.186879,0.642650
184,USA,0.858038,0.964735
185,KAZ,0.268187,0.073423


Gymnastics Logistic Regression

In [41]:
Gymnastics_df = data_df[data_df['Sport'] == "Gymnastics"]
Gymnastics_df = Gymnastics_df.dropna()

In [42]:
Gymnastics_dummies = pd.get_dummies(Gymnastics_df)
Gymnastics_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
85,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
86,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
87,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [43]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Gymnastics_dummies.drop(columns=['Medal'])
y_train = Gymnastics_dummies['Medal']

x_test = Gymnastics_dummies.drop(columns=['Medal'])
y_test = Gymnastics_dummies['Medal']

In [44]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9492940640655837

In [45]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Gymnastics_df)

array([0.23425397, 0.23425397, 0.23425397, ..., 0.19703156, 0.19703156,
       0.19703156])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
85,ESP,Gymnastics,False,0,1,0,1,8,0
86,ESP,Gymnastics,False,0,1,0,1,8,0
87,ESP,Gymnastics,False,0,1,0,1,8,0
88,ESP,Gymnastics,False,0,1,0,1,8,0
89,ESP,Gymnastics,False,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
203680,ITA,Gymnastics,False,0,0,0,3,7,0
203681,ITA,Gymnastics,False,0,0,0,3,7,0
203682,ITA,Gymnastics,False,0,0,0,3,7,0
203683,ITA,Gymnastics,False,0,0,0,3,7,0


In [46]:
Gymnastics_df['Predictions'] = y_pred.tolist()

In [47]:
Gymnastics_final = Gymnastics_df.drop_duplicates(subset=['NOC'], keep='last')
Gymnastics_final = Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2535500066.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Gymnastics_final = Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Cycling Logistic Regression

In [48]:
Cycling_df = data_df[data_df['Sport'] == "Cycling"]
Cycling_df = Cycling_df.dropna()

In [49]:
Cycling_dummies = pd.get_dummies(Cycling_df)
Cycling_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
131,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
132,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
173,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [50]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Cycling_dummies.drop(columns=['Medal'])
y_train = Cycling_dummies['Medal']

In [51]:
x_test = Cycling_dummies.drop(columns=['Medal'])
y_test = Cycling_dummies['Medal']

In [52]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9057317427989525

In [53]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Cycling_df)

array([0.54314685, 0.54314685, 0.04104853, ..., 0.58116293, 0.58116293,
       0.02388283])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
131,RUS,Cycling,True,0,0,0,2,6,0
132,RUS,Cycling,False,0,0,0,2,6,0
173,EGY,Cycling,False,0,0,0,5,7,0
204,USA,Cycling,False,0,2,1,1,8,5
241,EGY,Cycling,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203743,CHE,Cycling,False,0,2,0,2,8,0
203744,CHE,Cycling,False,0,2,0,2,8,0
203745,CHE,Cycling,False,0,1,0,2,8,0
203746,CHE,Cycling,False,0,1,0,2,8,0


In [54]:
Cycling_df['Predictions'] = y_pred.tolist()

In [55]:
Cycling_final = Cycling_df.drop_duplicates(subset=['NOC'], keep='last')
Cycling_final = Cycling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1199427753.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Cycling_final = Cycling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [56]:
B = pd.merge(Gymnastics_final, Cycling_final, how='outer', on=['NOC', 'NOC'])
B.rename(columns = {'Predictions_x':'Gymnastics', 'Predictions_y':'Cycling'}, inplace = True)
B = B.drop(["Sport_x","Sport_y"], 1)
B

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/597712745.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  B = B.drop(["Sport_x","Sport_y"], 1)


,NOC,Gymnastics,Cycling
0,YEM,0.018418,NaN
1,PAN,0.017619,NaN
2,TUN,0.021850,0.137492
3,BGD,0.021850,NaN
4,BRB,0.021850,0.003408
...,...,...,...
123,KHM,NaN,0.032304
124,HKG,NaN,0.063620
125,MWI,NaN,0.032304
126,LBN,NaN,0.104385


Rowing Logistic Regression

In [57]:
Rowing_df = data_df[data_df['Sport'] == "Rowing"]
Rowing_df = Rowing_df.dropna()

In [58]:
Rowing_dummies = pd.get_dummies(Rowing_df)
Rowing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
60,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
61,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
98,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [59]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Rowing_dummies.drop(columns=['Medal'])
y_train = Rowing_dummies['Medal']

x_test = Rowing_dummies.drop(columns=['Medal'])
y_test = Rowing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7811712846347607

In [60]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Rowing_df)

array([0.58952415, 0.58952415, 0.50795819, ..., 0.58952415, 0.38225132,
       0.38225132])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
60,NLD,Rowing,True,0,1,0,2,8,0
61,NLD,Rowing,False,0,1,0,2,8,0
98,ITA,Rowing,True,0,1,0,2,8,0
116,ARG,Rowing,False,0,0,0,0,7,0
130,CHL,Rowing,False,0,0,0,1,7,0
...,...,...,...,...,...,...,...,...,...
203971,NLD,Rowing,False,0,0,0,2,8,0
203972,NLD,Rowing,True,0,1,0,2,8,0
203973,NLD,Rowing,True,0,1,0,2,8,0
204025,RUS,Rowing,False,0,0,0,2,6,0


In [61]:
Rowing_df['Predictions'] = y_pred.tolist()
Rowing_final = Rowing_df.drop_duplicates(subset=['NOC'], keep='last')
Rowing_final = Rowing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


C:\Users\14043\AppData\Local\Temp/ipykernel_6344/772740510.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Rowing_final = Rowing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Shooting Logistic Regression

In [62]:
Shooting_df = data_df[data_df['Sport'] == "Shooting"]
Shooting_df = Shooting_df.dropna()

In [63]:
Shooting_dummies = pd.get_dummies(Shooting_df)
Shooting_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
79,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
124,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
137,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [64]:
x_train = Shooting_dummies.drop(columns=['Medal'])
y_train = Shooting_dummies['Medal']

x_test = Shooting_dummies.drop(columns=['Medal'])
y_test = Shooting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9352194180999839

In [65]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Shooting_df)

array([0.19086267, 0.2839292 , 0.2839292 , ..., 0.44243693, 0.19086267,
       0.19086267])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
79,NOR,Shooting,False,0,0,0,4,7,0
124,CHL,Shooting,False,0,0,0,1,7,0
137,CHL,Shooting,False,0,0,0,1,7,0
237,EGY,Shooting,False,0,0,0,5,7,0
282,EGY,Shooting,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203765,CHE,Shooting,False,0,2,0,2,8,0
203766,CHE,Shooting,False,0,2,0,2,8,0
203767,CHE,Shooting,False,0,2,0,2,8,0
203948,NOR,Shooting,False,0,0,0,4,7,0


In [66]:
Shooting_df['Predictions'] = y_pred.tolist()
Shooting_final = Shooting_df.drop_duplicates(subset=['NOC'], keep='last')
Shooting_final = Shooting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4008684553.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Shooting_final = Shooting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [67]:
C = pd.merge(Rowing_final, Shooting_final, how='outer', on=['NOC', 'NOC'])
C.rename(columns = {'Predictions_x':'Rowing', 'Predictions_y':'Shooting'}, inplace = True)
C = C.drop(["Sport_x","Sport_y"], 1)
C

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2485172735.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  C = C.drop(["Sport_x","Sport_y"], 1)


,NOC,Rowing,Shooting
0,KWT,0.191545,0.534135
1,TGO,0.008688,NaN
2,HND,0.091263,0.190863
3,TTO,0.186649,0.145883
4,CMR,0.103479,NaN
...,...,...,...
125,DOM,NaN,0.109677
126,BHR,NaN,0.216856
127,MAR,NaN,0.000967
128,MDA,NaN,0.646522


Fencing Logistic Regression

In [68]:
Fencing_df = data_df[data_df['Sport'] == "Fencing"]
Fencing_df = Fencing_df.dropna()
Fencing_df

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
99,ESP,Fencing,True,0,1,0,2,8,0
117,GRC,Fencing,False,0,1,0,2,8,0
141,MEX,Fencing,False,1,0,0,3,6,0
142,MEX,Fencing,False,1,0,0,3,6,0
143,MEX,Fencing,False,1,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
203312,AUT,Fencing,False,0,0,0,3,7,0
203313,AUT,Fencing,False,0,0,0,3,7,0
203408,CHN,Fencing,False,0,0,8,7,7,0
203409,CHN,Fencing,False,0,0,8,7,7,0


In [69]:
Fencing_dummies = pd.get_dummies(Fencing_df)
Fencing_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BEN,NOC_BFA,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
99,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
117,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
141,False,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [70]:
x_train = Fencing_dummies.drop(columns=['Medal'])
y_train = Fencing_dummies['Medal']

x_test = Fencing_dummies.drop(columns=['Medal'])
y_test = Fencing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8693467336683417

In [71]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Fencing_df)

array([0.09262811, 0.04416012, 0.05364159, ..., 0.12060436, 0.12060436,
       0.56247521])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
99,ESP,Fencing,True,0,1,0,2,8,0
117,GRC,Fencing,False,0,1,0,2,8,0
141,MEX,Fencing,False,1,0,0,3,6,0
142,MEX,Fencing,False,1,0,0,3,6,0
143,MEX,Fencing,False,1,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
203312,AUT,Fencing,False,0,0,0,3,7,0
203313,AUT,Fencing,False,0,0,0,3,7,0
203408,CHN,Fencing,False,0,0,8,7,7,0
203409,CHN,Fencing,False,0,0,8,7,7,0


In [72]:
Fencing_df['Predictions'] = y_pred.tolist()
Fencing_final = Fencing_df.drop_duplicates(subset=['NOC'], keep='last')
Fencing_final = Fencing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2959214136.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Fencing_final = Fencing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Canoeing Logistic Regression

In [73]:
Canoeing_df = data_df[data_df['Sport'] == "Canoeing"]
Canoeing_df = Canoeing_df.dropna()

In [74]:
Canoeing_dummies = pd.get_dummies(Canoeing_df)
Canoeing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
109,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
110,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
111,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [75]:
x_train = Canoeing_dummies.drop(columns=['Medal'])
y_train = Canoeing_dummies['Medal']

x_test = Canoeing_dummies.drop(columns=['Medal'])
y_test = Canoeing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8574193548387097

In [76]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Canoeing_df)

array([0.32177297, 0.32177297, 0.75110505, ..., 0.33039962, 0.33039962,
       0.39589379])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
109,CUB,Canoeing,False,0,0,0,2,7,0
110,CUB,Canoeing,False,0,0,0,2,7,0
111,BLR,Canoeing,False,0,0,0,2,7,0
112,BLR,Canoeing,True,0,0,0,2,7,0
203,CAN,Canoeing,False,1,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203817,USA,Canoeing,False,0,1,1,1,8,6
203818,USA,Canoeing,False,0,1,1,1,8,6
203819,USA,Canoeing,False,0,2,1,1,8,5
203820,USA,Canoeing,False,0,2,1,1,8,5


In [77]:
Canoeing_df['Predictions'] = y_pred.tolist()
Canoeing_final = Canoeing_df.drop_duplicates(subset=['NOC'], keep='last')
Canoeing_final = Canoeing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2271154440.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Canoeing_final = Canoeing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [78]:
D = pd.merge(Fencing_final, Canoeing_final, how='outer', on=['NOC', 'NOC'])
D.rename(columns = {'Predictions_x':'Fencing', 'Predictions_y':'Canoeing'}, inplace = True)
D = D.drop(["Sport_x","Sport_y"], 1)
D

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/242490794.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  D = D.drop(["Sport_x","Sport_y"], 1)


,NOC,Fencing,Canoeing
0,QAT,0.033488,NaN
1,BEN,0.060601,NaN
2,PRY,0.060601,NaN
3,PER,0.043281,0.240120
4,SAU,0.060601,NaN
...,...,...,...
98,MKD,NaN,0.317243
99,SVK,NaN,0.746868
100,SRB,NaN,0.233500
101,SVN,NaN,0.280143


Wrestling Logistic Regression

In [79]:
Wrestling_df = data_df[data_df['Sport'] == "Wrestling"]
Wrestling_df = Wrestling_df.dropna()

In [80]:
Wrestling_dummies = pd.get_dummies(Wrestling_df)
Wrestling_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_ALB,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
52,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
53,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
84,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [81]:
x_train = Wrestling_dummies.drop(columns=['Medal'])
y_train = Wrestling_dummies['Medal']

x_test = Wrestling_dummies.drop(columns=['Medal'])
y_test = Wrestling_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.861604997597309

In [82]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Wrestling_df)

array([0.37197127, 0.37197127, 0.22172756, ..., 0.76726394, 0.18305344,
       0.49652793])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
52,NOR,Wrestling,False,0,1,0,2,8,0
53,NOR,Wrestling,False,0,2,0,2,8,0
84,EST,Wrestling,False,0,0,0,2,7,0
125,USA,Wrestling,True,0,1,1,1,8,6
207,EGY,Wrestling,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203710,HRV,Wrestling,False,0,0,0,3,8,0
203711,HRV,Wrestling,False,0,0,0,3,8,0
203779,USA,Wrestling,False,1,1,1,2,8,5
203924,LVA,Wrestling,False,0,0,0,3,7,0


In [83]:
Wrestling_df['Predictions'] = y_pred.tolist()
Wrestling_final = Wrestling_df.drop_duplicates(subset=['NOC'], keep='last')
Wrestling_final = Wrestling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1911261898.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Wrestling_final = Wrestling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Football Logistic Regression

In [84]:
Football_df = data_df[data_df['Sport'] == "Football"]
Football_df = Football_df.dropna()
 
print(Football_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
69      NOR  Football   True     0           1              0             2   
118     CMR  Football   True     0           0              0             5   
122     GRC  Football  False     0           1              0             2   
134     ITA  Football  False     0           1              0             2   
166     IRQ  Football  False     0           0              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203675  SAU  Football  False     0           0              0             1   
203725  NLD  Football  False     0           2              0             1   
203784  ESP  Football  False     0           0              0             2   
203859  CAN  Football  False     0           2              0             1   
204024  USA  Football  False     0           0              2             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
69    

In [85]:
Football_dummies = pd.get_dummies(Football_df)
Football_dummies.head(3)

,Medal,Host,NOC_ARE,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BLR,NOC_BRA,NOC_CAN,NOC_CHE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
69,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
118,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
122,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [86]:
x_train = Football_dummies.drop(columns=['Medal'])
y_train = Football_dummies['Medal']

x_test = Football_dummies.drop(columns=['Medal'])
y_test = Football_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8740793537657401

In [87]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Football_df)

array([0.89866694, 0.80528335, 0.04849377, ..., 0.29958555, 0.91032369,
       0.01981636])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
69,NOR,Football,True,0,1,0,2,8,0
118,CMR,Football,True,0,0,0,5,5,0
122,GRC,Football,False,0,1,0,2,8,0
134,ITA,Football,False,0,1,0,2,8,0
166,IRQ,Football,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203675,SAU,Football,False,0,0,0,1,7,0
203725,NLD,Football,False,0,2,0,1,8,0
203784,ESP,Football,False,0,0,0,2,7,0
203859,CAN,Football,False,0,2,0,1,8,0


In [88]:
Football_df['Predictions'] = y_pred.tolist()
Football_final = Football_df.drop_duplicates(subset=['NOC'], keep='last')
Football_final = Football_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3633843560.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Football_final = Football_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [89]:
E = pd.merge(Wrestling_final, Football_final, how='outer', on=['NOC', 'NOC'])
E.rename(columns = {'Predictions_x':'Wrestling', 'Predictions_y':'Football'}, inplace = True)
E = E.drop(["Sport_x","Sport_y"], 1)
E

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3963166347.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  E = E.drop(["Sport_x","Sport_y"], 1)


,NOC,Wrestling,Football
0,YEM,0.024007,NaN
1,CHL,0.196642,0.892595
2,BHR,0.298731,NaN
3,LBN,0.165726,NaN
4,CIV,0.050638,0.000010
...,...,...,...
123,CRI,NaN,0.026629
124,GHA,NaN,0.139544
125,PRY,NaN,0.807399
126,KWT,NaN,0.022987


Boxing Logistic Regression

In [90]:
Boxing_df = data_df[data_df['Sport'] == "Boxing"]
Boxing_df = Boxing_df.dropna()


In [91]:
Boxing_dummies = pd.get_dummies(Boxing_df)
Boxing_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
80,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
94,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
102,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [92]:
x_train = Boxing_dummies.drop(columns=['Medal'])
y_train = Boxing_dummies['Medal']

x_test = Boxing_dummies.drop(columns=['Medal'])
y_test = Boxing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8773800456968773

In [93]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Boxing_df)

array([0.29804674, 0.62642901, 0.04649243, ..., 0.64030124, 0.34153902,
       0.34153902])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
80,MAR,Boxing,False,0,0,0,3,7,0
94,BGR,Boxing,False,0,0,0,3,7,0
102,SDN,Boxing,False,0,0,0,7,5,0
119,CMR,Boxing,False,0,0,0,7,4,0
145,NGA,Boxing,False,0,0,0,7,4,0
...,...,...,...,...,...,...,...,...,...
203780,CHL,Boxing,False,0,0,0,2,6,0
203788,COL,Boxing,False,0,0,0,3,7,0
203857,ITA,Boxing,False,0,0,0,3,7,0
203877,BLR,Boxing,True,0,0,0,2,7,0


In [94]:
Boxing_df['Predictions'] = y_pred.tolist()
Boxing_final = Boxing_df.drop_duplicates(subset=['NOC'], keep='last')
Boxing_final = Boxing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4182983223.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Boxing_final = Boxing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Equestrianism Logistic Regression

In [95]:
Equestrianism_df = data_df[data_df['Sport'] == "Equestrianism"]
Equestrianism_df = Equestrianism_df.dropna()

In [96]:
Equestrianism_dummies = pd.get_dummies(Equestrianism_df)
Equestrianism_dummies.head(3)

,Medal,Host,NOC_ARE,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BLR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
73,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
196,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
197,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [97]:
x_train = Equestrianism_dummies.drop(columns=['Medal'])
y_train = Equestrianism_dummies['Medal']

x_test = Equestrianism_dummies.drop(columns=['Medal'])
y_test = Equestrianism_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8697627460878344

In [98]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Equestrianism_df)

array([0.07933694, 0.05436971, 0.05436971, ..., 0.19633813, 0.19633813,
       0.19633813])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
73,NOR,Equestrianism,False,0,1,0,2,8,0
196,IRL,Equestrianism,False,0,2,0,3,8,0
197,IRL,Equestrianism,False,0,2,0,3,8,0
1037,ARG,Equestrianism,False,0,0,0,2,6,0
1038,ARG,Equestrianism,False,0,0,0,2,6,0
...,...,...,...,...,...,...,...,...,...
203407,ARG,Equestrianism,False,0,0,0,1,7,0
203598,DNK,Equestrianism,False,0,2,0,1,8,0
203599,DNK,Equestrianism,True,0,2,0,1,8,0
203600,DNK,Equestrianism,False,0,2,0,1,8,0


In [99]:
Equestrianism_df['Predictions'] = y_pred.tolist()
Equestrianism_final = Equestrianism_df.drop_duplicates(subset=['NOC'], keep='last')
Equestrianism_final = Equestrianism_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1249077138.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Equestrianism_final = Equestrianism_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [100]:
F = pd.merge(Boxing_final, Equestrianism_final, how='outer', on=['NOC', 'NOC'])
F.rename(columns = {'Predictions_x':'Boxing', 'Predictions_y':'Equestrianism'}, inplace = True)
F = F.drop(["Sport_x","Sport_y"], 1)
F

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1904098448.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  F = F.drop(["Sport_x","Sport_y"], 1)


,NOC,Boxing,Equestrianism
0,OMN,0.270816,NaN
1,KWT,0.217035,NaN
2,SLB,0.133539,NaN
3,BOL,0.133539,0.041492
4,FSM,0.094152,NaN
...,...,...,...
147,COL,0.432409,0.075766
148,ITA,0.640301,0.414024
149,BLR,0.341539,0.112333
150,ARE,NaN,0.124626


Judo Logistic Regression

In [101]:
Judo_df = data_df[data_df['Sport'] == "Judo"]
Judo_df = Judo_df.dropna()

In [102]:
Judo_dummies = pd.get_dummies(Judo_df)
Judo_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
1,False,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
100,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
120,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [103]:
x_train = Judo_dummies.drop(columns=['Medal'])
y_train = Judo_dummies['Medal']

x_test = Judo_dummies.drop(columns=['Medal'])
y_test = Judo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8726263511539585

In [104]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Judo_df)

array([0.64518974, 0.00319843, 0.29463751, ..., 0.57990838, 0.69659866,
       0.64377745])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
1,CHN,Judo,False,0,0,7,4,7,2
100,TCD,Judo,False,0,0,0,7,4,0
120,TUR,Judo,False,0,0,0,3,7,0
300,EGY,Judo,False,0,0,0,5,7,0
314,EGY,Judo,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203810,CAN,Judo,False,0,2,0,1,8,0
203811,CAN,Judo,False,0,1,0,1,8,0
203922,GEO,Judo,True,0,0,0,4,7,0
203960,NLD,Judo,True,0,1,0,2,8,0


In [105]:
Judo_df['Predictions'] = y_pred.tolist()
Judo_final = Judo_df.drop_duplicates(subset=['NOC'], keep='last')
Judo_final = Judo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/405281857.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Judo_final = Judo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Basketball Logistic Regression

In [106]:
Basketball_df = data_df[data_df['Sport'] == "Basketball"]
Basketball_df = Basketball_df.dropna()

In [107]:
Basketball_dummies = pd.get_dummies(Basketball_df)
Basketball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAF,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
0,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
107,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
192,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [108]:
x_train = Basketball_dummies.drop(columns=['Medal'])
y_train = Basketball_dummies['Medal']

x_test = Basketball_dummies.drop(columns=['Medal'])
y_test = Basketball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8938140919616275

In [109]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Basketball_df)

array([0.47124788, 0.8456451 , 0.26250841, ..., 0.61612327, 0.00209901,
       0.01371066])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0,8,6,7,0
107,ESP,Basketball,False,0,1,0,2,8,0
192,ITA,Basketball,False,0,0,0,3,8,0
257,EGY,Basketball,False,0,0,0,5,5,0
264,EGY,Basketball,False,0,0,0,5,6,0
...,...,...,...,...,...,...,...,...,...
203733,LTU,Basketball,False,0,0,0,3,7,0
203734,LTU,Basketball,True,0,0,0,3,7,0
203735,LTU,Basketball,False,0,0,0,3,7,0
203956,DEU,Basketball,False,0,2,0,2,8,1


In [110]:
Basketball_df['Predictions'] = y_pred.tolist()
Basketball_final = Basketball_df.drop_duplicates(subset=['NOC'], keep='last')
Basketball_final = Basketball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1911532976.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Basketball_final = Basketball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [111]:
G = pd.merge(Judo_final, Basketball_final, how='outer', on=['NOC', 'NOC'])
G.rename(columns = {'Predictions_x':'Judo', 'Predictions_y':'Basketball'}, inplace = True)
G = G.drop(["Sport_x","Sport_y"], 1)
G

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3586753422.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  G = G.drop(["Sport_x","Sport_y"], 1)


,NOC,Judo,Basketball
0,GUM,0.133953,NaN
1,TGO,0.068391,NaN
2,NIC,0.180088,NaN
3,TTO,0.180088,NaN
4,BEN,0.068391,NaN
...,...,...,...
152,HUN,0.430879,NaN
153,CHL,0.174663,NaN
154,CAN,0.194537,0.002087
155,GEO,0.579908,NaN


Handball Logistic Regression

In [112]:
Handball_df = data_df[data_df['Sport'] == "Handball"]
Handball_df = Handball_df.dropna()

In [113]:
Handball_dummies = pd.get_dummies(Handball_df)
Handball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
50,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
113,True,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
114,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [114]:
x_train = Handball_dummies.drop(columns=['Medal'])
y_train = Handball_dummies['Medal']

x_test = Handball_dummies.drop(columns=['Medal'])
y_test = Handball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.7936344969199178

In [115]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Handball_df)

array([0.98720262, 0.58508325, 0.75117675, ..., 0.51196815, 0.02500933,
       0.02364176])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
50,NOR,Handball,True,0,4,0,2,8,0
113,FRA,Handball,True,0,2,0,2,8,1
114,FRA,Handball,True,0,1,0,1,8,0
115,FRA,Handball,True,0,1,0,1,8,0
135,FRA,Handball,False,0,1,0,2,8,1
...,...,...,...,...,...,...,...,...,...
203758,CUB,Handball,False,0,0,0,2,7,0
203770,CHE,Handball,False,0,2,0,2,8,0
203787,ESP,Handball,False,0,0,0,2,8,0
203858,AUT,Handball,False,0,1,0,3,7,0


In [116]:
Handball_df['Predictions'] = y_pred.tolist()
Handball_final = Handball_df.drop_duplicates(subset=['NOC'], keep='last')
Handball_final = Handball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1399169446.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Handball_final = Handball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Volleyball

In [117]:
Volleyball_df = data_df[data_df['Sport'] == "Volleyball"]
Volleyball_df = Volleyball_df.dropna()

In [118]:
Volleyball_dummies = pd.get_dummies(Volleyball_df)
Volleyball_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CMR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
210,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
218,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
236,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [119]:
x_train = Volleyball_dummies.drop(columns=['Medal'])
y_train = Volleyball_dummies['Medal']

x_test = Volleyball_dummies.drop(columns=['Medal'])
y_test = Volleyball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8166845685642679

In [120]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Volleyball_df)

array([0.03064459, 0.03064459, 0.03064459, ..., 0.42619056, 0.42619056,
       0.01264008])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
210,EGY,Volleyball,False,0,0,0,5,7,0
218,EGY,Volleyball,False,0,0,0,5,7,0
236,EGY,Volleyball,False,0,0,0,5,7,0
254,EGY,Volleyball,False,0,0,0,5,6,0
270,EGY,Volleyball,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203795,CHN,Volleyball,False,0,0,8,7,7,0
203945,NLD,Volleyball,False,0,0,0,2,8,0
203946,NLD,Volleyball,True,0,1,0,2,8,0
203947,NLD,Volleyball,True,0,1,0,2,8,0


In [121]:
Volleyball_df['Predictions'] = y_pred.tolist()
Volleyball_final = Volleyball_df.drop_duplicates(subset=['NOC'], keep='last')
Volleyball_final = Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3185448111.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Volleyball_final = Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [122]:
H = pd.merge(Handball_final, Volleyball_final, how='outer', on=['NOC', 'NOC'])
H.rename(columns = {'Predictions_x':'Handball', 'Predictions_y':'Volleyball'}, inplace = True)
H = H.drop(["Sport_x","Sport_y"], 1)
H

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1895146923.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  H = H.drop(["Sport_x","Sport_y"], 1)


,NOC,Handball,Volleyball
0,COG,0.057434,NaN
1,KWT,0.011341,NaN
2,DZA,0.007168,0.015416
3,AUS,0.051625,0.034005
4,ISL,0.478190,NaN
5,NOR,0.481305,NaN
6,CZE,0.011341,NaN
7,CAN,0.014531,0.018648
8,ARG,0.025164,0.450394
9,CIV,0.009623,NaN


Weightlifting

In [123]:
Weightlifting_df = data_df[data_df['Sport'] == "Weightlifting"]
Weightlifting_df = Weightlifting_df.dropna()

In [124]:
Weightlifting_dummies = pd.get_dummies(Weightlifting_df)
Weightlifting_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
51,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
165,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
176,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [125]:
x_train = Weightlifting_dummies.drop(columns=['Medal'])
y_train = Weightlifting_dummies['Medal']

x_test = Weightlifting_dummies.drop(columns=['Medal'])
y_test = Weightlifting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8793300342596118

In [126]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Weightlifting_df)

array([0.46818908, 0.12140571, 0.44994663, ..., 0.24982521, 0.33858121,
       0.65621903])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
51,ROU,Weightlifting,False,0,0,0,4,7,0
165,IRQ,Weightlifting,False,0,0,0,4,5,0
176,QAT,Weightlifting,False,0,1,0,0,8,0
188,AZE,Weightlifting,False,0,0,0,4,7,0
195,IRQ,Weightlifting,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203502,MAR,Weightlifting,False,0,0,0,4,7,0
203658,CZE,Weightlifting,False,0,0,0,2,7,0
203855,ESP,Weightlifting,False,0,0,0,2,8,0
203856,ESP,Weightlifting,False,1,0,0,2,8,0


In [127]:
Weightlifting_df['Predictions'] = y_pred.tolist()
Weightlifting_final = Weightlifting_df.drop_duplicates(subset=['NOC'], keep='last')
Weightlifting_final = Weightlifting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1673677086.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Weightlifting_final = Weightlifting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Water Polo

In [128]:
Polo_df = data_df[data_df['Sport'] == "Water Polo"]
Polo_df = Polo_df.dropna()

In [129]:
Polo_dummies = pd.get_dummies(Polo_df)
Polo_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CUB,NOC_CZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
58,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
121,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
189,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [130]:
x_train = Polo_dummies.drop(columns=['Medal'])
y_train = Polo_dummies['Medal']

x_test = Polo_dummies.drop(columns=['Medal'])
y_test = Polo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8627450980392157

In [131]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Polo_df)

array([0.17290969, 0.18490344, 0.67229338, ..., 0.86623266, 0.00853121,
       0.30006642])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
58,NLD,Water Polo,False,0,0,0,3,8,0
121,ESP,Water Polo,True,0,0,0,2,8,0
189,ITA,Water Polo,False,0,1,0,2,8,0
251,EGY,Water Polo,False,0,0,0,5,7,0
267,EGY,Water Polo,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203275,RUS,Water Polo,False,0,0,0,2,6,0
203613,ESP,Water Polo,False,0,0,0,3,7,0
203630,RUS,Water Polo,False,0,0,0,2,6,0
203631,KAZ,Water Polo,False,0,0,0,4,6,0


In [132]:
Polo_df['Predictions'] = y_pred.tolist()
Polo_final = Polo_df.drop_duplicates(subset=['NOC'], keep='last')
Polo_final = Polo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1686184237.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Polo_final = Polo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [133]:
I = pd.merge(Weightlifting_final, Polo_final, how='outer', on=['NOC', 'NOC'])
I.rename(columns = {'Predictions_x':'Weightlifting', 'Predictions_y':'Water Polo'}, inplace = True)
I = I.drop(["Sport_x","Sport_y"], 1)
I

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1958365582.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  I = I.drop(["Sport_x","Sport_y"], 1)


,NOC,Weightlifting,Water Polo
0,SAU,0.199777,NaN
1,GHA,0.055787,NaN
2,ZWE,0.018350,NaN
3,SLE,0.060117,NaN
4,LBN,0.199777,NaN
...,...,...,...
121,CZE,0.144086,0.016830
122,ESP,0.338581,0.479415
123,POL,0.656219,NaN
124,SRB,NaN,0.988491


Archery

In [134]:
Archery_df = data_df[data_df['Sport'] == "Archery"]
Archery_df = Archery_df.dropna()

In [135]:
Archery_dummies = pd.get_dummies(Archery_df)
Archery_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,NOC_BLR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
206,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
521,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
764,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [136]:
x_train = Archery_dummies.drop(columns=['Medal'])
y_train = Archery_dummies['Medal']

x_test = Archery_dummies.drop(columns=['Medal'])
y_test = Archery_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9129082426127527

In [137]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Archery_df)

array([0.01309408, 0.05045681, 0.51950213, ..., 0.82801138, 0.82801138,
       0.21285733])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
206,MAR,Archery,False,0,0,0,4,7,0
521,EGY,Archery,False,0,0,0,5,7,0
764,RUS,Archery,False,0,0,0,2,7,0
765,RUS,Archery,False,0,0,0,2,7,0
845,IRN,Archery,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
202043,CHN,Archery,False,0,0,8,5,7,1
202044,CHN,Archery,True,0,0,8,5,7,1
202045,CHN,Archery,True,1,0,7,4,7,1
202046,CHN,Archery,True,1,0,7,4,7,1


In [138]:
Archery_df['Predictions'] = y_pred.tolist()
Archery_final = Archery_df.drop_duplicates(subset=['NOC'], keep='last')
Archery_final = Archery_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4291309299.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Archery_final = Archery_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Diving

In [139]:
Diving_df = data_df[data_df['Sport'] == "Diving"]
Diving_df = Diving_df.dropna()

In [140]:
Diving_dummies = pd.get_dummies(Diving_df)
Diving_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
108,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
235,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
538,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [141]:
x_train = Diving_dummies.drop(columns=['Medal'])
y_train = Diving_dummies['Medal']

x_test = Diving_dummies.drop(columns=['Medal'])
y_test = Diving_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8816234498308907

In [142]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Diving_df)

array([0.0337998 , 0.04389838, 0.51642529, ..., 0.25327156, 0.25327156,
       0.25327156])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
108,CUB,Diving,False,0,0,0,2,8,0
235,EGY,Diving,False,0,0,0,5,7,0
538,CAN,Diving,False,0,2,0,1,8,0
539,CAN,Diving,False,0,2,0,1,8,0
540,CAN,Diving,True,0,2,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202802,UKR,Diving,False,0,0,0,3,7,0
202803,UKR,Diving,False,0,0,0,3,7,0
202804,UKR,Diving,False,0,0,0,3,7,0
202805,UKR,Diving,True,0,0,0,3,7,0


In [143]:
Diving_df['Predictions'] = y_pred.tolist()
Diving_final = Diving_df.drop_duplicates(subset=['NOC'], keep='last')
Diving_final = Diving_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1671132766.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Diving_final = Diving_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [144]:
J = pd.merge(Archery_final, Diving_final, how='outer', on=['NOC', 'NOC'])
J.rename(columns = {'Predictions_x':'Archery', 'Predictions_y':'Diving'}, inplace = True)
J = J.drop(["Sport_x","Sport_y"], 1)
J

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2986169119.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  J = J.drop(["Sport_x","Sport_y"], 1)


,NOC,Archery,Diving
0,MAR,0.013094,NaN
1,IRQ,0.050457,NaN
2,JOR,0.050457,NaN
3,ROU,0.013094,0.043898
4,DOM,0.031316,0.037720
...,...,...,...
93,BHS,NaN,0.043898
94,PER,NaN,0.043898
95,SYR,NaN,0.043898
96,ECU,NaN,0.037720


Tennis

In [145]:
Tennis_df = data_df[data_df['Sport'] == "Tennis"]
Tennis_df = Tennis_df.dropna()

In [146]:
Tennis_dummies = pd.get_dummies(Tennis_df)
Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BEN,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
123,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
446,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1008,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [147]:
x_train = Tennis_dummies.drop(columns=['Medal'])
y_train = Tennis_dummies['Medal']

x_test = Tennis_dummies.drop(columns=['Medal'])
y_test = Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9209989806320081

In [148]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Tennis_df)

array([0.06810324, 0.02988844, 0.02199308, ..., 0.67155865, 0.67155865,
       0.67155865])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
123,MEX,Tennis,False,0,0,0,2,7,0
446,NGA,Tennis,False,0,0,0,6,4,0
1008,BRA,Tennis,False,0,0,1,2,6,0
1178,ZAF,Tennis,False,0,0,0,4,5,0
1607,USA,Tennis,True,1,1,1,2,8,5
...,...,...,...,...,...,...,...,...,...
203920,BLR,Tennis,False,0,0,0,2,7,0
203921,BLR,Tennis,False,0,0,0,2,7,0
203926,RUS,Tennis,True,0,0,0,2,7,0
203927,RUS,Tennis,False,0,0,0,2,7,0


In [149]:
Tennis_df['Predictions'] = y_pred.tolist()
Tennis_final = Tennis_df.drop_duplicates(subset=['NOC'], keep='last')
Tennis_final = Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Tennis_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3496524252.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Tennis_final = Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
3362,MDA,Tennis,0.167711
7152,SLV,Tennis,0.042138
15122,LTU,Tennis,0.042138
24797,TUR,Tennis,0.117731
25595,BOL,Tennis,0.030837
...,...,...,...
203567,CZE,Tennis,0.575507
203569,SVK,Tennis,0.036352
203757,COL,Tennis,0.117731
203921,BLR,Tennis,0.538288


Table Tennis

In [150]:
Table_Tennis_df = data_df[data_df['Sport'] == "Table Tennis"]
Table_Tennis_df = Table_Tennis_df.dropna()

In [151]:
Table_Tennis_dummies = pd.get_dummies(Table_Tennis_df)
Table_Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BIH,NOC_BLR,NOC_BRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
369,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
370,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
371,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [152]:
x_train = Table_Tennis_dummies.drop(columns=['Medal'])
y_train = Table_Tennis_dummies['Medal']

x_test = Table_Tennis_dummies.drop(columns=['Medal'])
y_test = Table_Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9215572341661824

In [153]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Table_Tennis_df)

array([0.00587437, 0.00587437, 0.00587437, ..., 0.00535706, 0.00567511,
       0.00664753])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
369,EGY,Table Tennis,False,0,0,0,5,7,0
370,EGY,Table Tennis,False,0,0,0,5,7,0
371,EGY,Table Tennis,False,0,0,0,5,7,0
611,NGA,Table Tennis,False,0,0,1,4,5,0
962,IND,Table Tennis,False,0,0,7,6,6,0
...,...,...,...,...,...,...,...,...,...
202757,USA,Table Tennis,False,1,1,1,2,8,5
202758,USA,Table Tennis,False,0,1,1,1,8,6
202759,USA,Table Tennis,False,0,1,1,1,8,6
202760,USA,Table Tennis,False,0,2,1,1,8,5


In [154]:
Table_Tennis_df['Predictions'] = y_pred.tolist()
Table_Tennis_final = Table_Tennis_df.drop_duplicates(subset=['NOC'], keep='last')
Table_Tennis_final = Table_Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/959563229.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Table_Tennis_final = Table_Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [155]:
K = pd.merge(Tennis_final, Table_Tennis_final, how='outer', on=['NOC', 'NOC'])
K.rename(columns = {'Predictions_x':'Tennis', 'Predictions_y':'Table Tennis'}, inplace = True)
K = K.drop(["Sport_x","Sport_y"], 1)
K

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2974582299.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  K = K.drop(["Sport_x","Sport_y"], 1)


,NOC,Tennis,Table Tennis
0,MDA,0.167711,NaN
1,SLV,0.042138,NaN
2,LTU,0.042138,0.006648
3,TUR,0.117731,0.006648
4,BOL,0.030837,NaN
...,...,...,...
106,VUT,NaN,0.003066
107,HKG,NaN,0.107383
108,COG,NaN,0.007993
109,FJI,NaN,0.006648


Badminton

In [156]:
Badminton_df = data_df[data_df['Sport'] == "Badminton"]
Badminton_df = Badminton_df.dropna()

In [157]:
Badminton_dummies = pd.get_dummies(Badminton_df)
Badminton_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_BRN,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
26,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
593,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
594,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [158]:
x_train = Badminton_dummies.drop(columns=['Medal'])
y_train = Badminton_dummies['Medal']

x_test = Badminton_dummies.drop(columns=['Medal'])
y_test = Badminton_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8830242510699001

In [159]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Badminton_df)

array([0.03055753, 0.22673144, 0.36219098, ..., 0.02278136, 0.02278136,
       0.01241942])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
26,FIN,Badminton,False,0,1,0,1,8,0
593,ESP,Badminton,False,0,1,0,2,8,0
594,ESP,Badminton,False,0,1,0,1,8,0
595,ESP,Badminton,False,0,1,0,1,8,0
1239,GBR,Badminton,False,1,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
203867,RUS,Badminton,False,0,0,1,2,6,0
203868,RUS,Badminton,False,0,0,0,2,6,0
203952,DEU,Badminton,False,0,2,0,2,8,1
203953,DEU,Badminton,False,0,2,0,2,8,1


In [160]:
Badminton_df['Predictions'] = y_pred.tolist()
Badminton_final = Badminton_df.drop_duplicates(subset=['NOC'], keep='last')
Badminton_final = Badminton_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1196665366.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Badminton_final = Badminton_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Modern Pentathlon

In [161]:
Pentathlon_df = data_df[data_df['Sport'] == "Modern Pentathlon"]
Pentathlon_df = Pentathlon_df.dropna()

In [162]:
Pentathlon_dummies = pd.get_dummies(Pentathlon_df)
Pentathlon_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BHR,NOC_BLR,NOC_BRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
355,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
356,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
357,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [163]:
x_train = Pentathlon_dummies.drop(columns=['Medal'])
y_train = Pentathlon_dummies['Medal']

x_test = Pentathlon_dummies.drop(columns=['Medal'])
y_test = Pentathlon_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9207135777998018

In [164]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Pentathlon_df)

array([0.03069719, 0.03808628, 0.03808628, ..., 0.16515953, 0.05436039,
       0.05436039])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
355,EGY,Modern Pentathlon,False,0,0,0,5,7,0
356,EGY,Modern Pentathlon,False,0,0,0,5,6,0
357,EGY,Modern Pentathlon,False,0,0,0,5,6,0
358,EGY,Modern Pentathlon,False,0,0,0,5,6,0
359,EGY,Modern Pentathlon,False,0,0,0,5,6,0
...,...,...,...,...,...,...,...,...,...
202201,CHN,Modern Pentathlon,False,0,0,7,4,8,3
202814,BLR,Modern Pentathlon,False,0,0,0,2,7,0
202989,DEU,Modern Pentathlon,False,0,1,0,2,8,1
203310,AUT,Modern Pentathlon,False,0,0,0,3,7,0


In [165]:
Pentathlon_df['Predictions'] = y_pred.tolist()
Pentathlon_final = Pentathlon_df.drop_duplicates(subset=['NOC'], keep='last')
Pentathlon_final = Pentathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1043166688.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Pentathlon_final = Pentathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [166]:
L = pd.merge(Badminton_final, Pentathlon_final, how='outer', on=['NOC', 'NOC'])
L.rename(columns = {'Predictions_x':'Badminton', 'Predictions_y':'Modern Pentathlon'}, inplace = True)
L = L.drop(["Sport_x","Sport_y"], 1)
L

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1863086147.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  L = L.drop(["Sport_x","Sport_y"], 1)


,NOC,Badminton,Modern Pentathlon
0,ROU,0.042557,0.070235
1,ITA,0.016015,0.298581
2,SYC,0.042557,NaN
3,UGA,0.049931,NaN
4,EGY,0.063675,0.038086
...,...,...,...
69,LVA,NaN,0.561836
70,KAZ,NaN,0.490176
71,KGZ,NaN,0.166414
72,CUB,NaN,0.097993


Synchronized Swimming

In [167]:
S_Swimming_df = data_df[data_df['Sport'] == "Synchronized Swimming"]
S_Swimming_df = S_Swimming_df.dropna()

In [168]:
S_Swimming_dummies = pd.get_dummies(S_Swimming_df)
S_Swimming_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
214,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
215,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
216,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [169]:
x_train = S_Swimming_dummies.drop(columns=['Medal'])
y_train = S_Swimming_dummies['Medal']

x_test = S_Swimming_dummies.drop(columns=['Medal'])
y_test = S_Swimming_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.905638665132336

In [170]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,S_Swimming_df)

array([2.00022442e-02, 2.00022442e-02, 2.00022442e-02, 2.00022442e-02,
       2.00022442e-02, 2.00022442e-02, 2.00022442e-02, 2.00022442e-02,
       2.00022442e-02, 2.00022442e-02, 2.00022442e-02, 3.23544910e-02,
       1.27280297e-02, 3.99654427e-02, 3.43859092e-02, 1.57401135e-01,
       1.57401135e-01, 1.57401135e-01, 2.00022442e-02, 7.32937860e-02,
       2.00022442e-02, 2.00022442e-02, 7.47653588e-01, 2.67920507e-04,
       2.67920507e-04, 2.00022442e-02, 2.00022442e-02, 2.00022442e-02,
       3.67483328e-04, 3.56010090e-02, 3.56010090e-02, 3.11016804e-02,
       3.23544910e-02, 2.00022442e-02, 2.00022442e-02, 1.96257505e-04,
       1.27280297e-02, 1.27280297e-02, 1.27280297e-02, 5.24907402e-01,
       5.59585215e-01, 5.59585215e-01, 2.00022442e-02, 2.00022442e-02,
       4.83622592e-02, 2.00022442e-02, 1.27280297e-02, 5.24907402e-01,
       5.24907402e-01, 4.56735829e-01, 5.25990692e-01, 2.00022442e-02,
       5.92702611e-02, 5.24907402e-01, 5.24907402e-01, 5.59585215e-01,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
214,EGY,Synchronized Swimming,False,0,0,0,5,7,0
215,EGY,Synchronized Swimming,False,0,0,0,5,7,0
216,EGY,Synchronized Swimming,False,0,0,0,5,7,0
246,EGY,Synchronized Swimming,False,0,0,0,5,7,0
247,EGY,Synchronized Swimming,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
202196,CHN,Synchronized Swimming,True,1,0,7,4,7,1
202249,CHN,Synchronized Swimming,False,0,0,8,7,7,0
202444,KAZ,Synchronized Swimming,False,0,0,0,4,7,0
203935,NLD,Synchronized Swimming,False,0,1,0,2,8,0


In [171]:
S_Swimming_df['Predictions'] = y_pred.tolist()
S_Swimming_final = S_Swimming_df.drop_duplicates(subset=['NOC'], keep='last')
S_Swimming_final = S_Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
S_Swimming_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1902890597.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  S_Swimming_final = S_Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
6909,COL,Synchronized Swimming,0.020002
78326,SWE,Synchronized Swimming,0.031102
97996,SVK,Synchronized Swimming,0.020002
99981,FIN,Synchronized Swimming,0.031102
112650,HUN,Synchronized Swimming,0.017296
153259,CUB,Synchronized Swimming,0.020002
154914,VEN,Synchronized Swimming,0.012728
156097,NZL,Synchronized Swimming,0.012728
157764,ARG,Synchronized Swimming,0.073294
161006,DEU,Synchronized Swimming,0.086904


Baseball

In [172]:
Baseball_df = data_df[data_df['Sport'] == "Baseball"]
Baseball_df = Baseball_df.dropna()

In [173]:
Baseball_dummies = pd.get_dummies(Baseball_df)
Baseball_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_CAN,NOC_CHN,NOC_CUB,NOC_DOM,NOC_ESP,NOC_GRC,NOC_ITA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
505,False,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
506,False,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
578,True,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [174]:
x_train = Baseball_dummies.drop(columns=['Medal'])
y_train = Baseball_dummies['Medal']

x_test = Baseball_dummies.drop(columns=['Medal'])
y_test = Baseball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9463087248322147

In [175]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Baseball_df)

array([2.06465954e-01, 3.41876616e-02, 9.68680008e-01, 2.29965400e-01,
       7.30959231e-03, 2.53873917e-02, 6.48336331e-04, 4.67237007e-03,
       9.68638521e-01, 9.68680008e-01, 9.81214064e-01, 9.81214064e-01,
       9.99967103e-01, 2.06465954e-01, 3.96136290e-04, 7.30959231e-03,
       7.30959231e-03, 2.29965400e-01, 9.27033443e-01, 6.44983974e-01,
       9.69060555e-01, 1.00294342e-01, 6.03953714e-01, 9.68638521e-01,
       3.41876616e-02, 3.96136290e-04, 9.99967103e-01, 3.41876616e-02,
       3.41876616e-02, 3.96136290e-04, 6.23068646e-02, 9.69060555e-01,
       9.81214064e-01, 6.23068646e-02, 3.96136290e-04, 4.54138535e-02,
       3.96136290e-04, 6.44983974e-01, 3.76481880e-03, 5.53856376e-03,
       3.87382527e-02, 6.48336331e-04, 2.53873917e-02, 2.53873917e-02,
       6.48336331e-04, 1.00294342e-01, 5.53856376e-03, 9.69060555e-01,
       6.23068646e-02, 1.00294342e-01, 6.53187716e-03, 4.54138535e-02,
       2.53873917e-02, 6.48336331e-04, 9.99967103e-01, 7.30959231e-03,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
505,JPN,Baseball,False,0,1,0,1,8,3
506,JPN,Baseball,False,0,1,0,1,8,1
578,USA,Baseball,True,0,1,1,1,8,6
1216,USA,Baseball,False,0,1,1,2,7,5
1439,ZAF,Baseball,False,0,0,0,4,5,0
...,...,...,...,...,...,...,...,...,...
201091,NIC,Baseball,False,0,0,0,4,7,0
201419,GRC,Baseball,False,0,1,0,2,8,0
202009,CHN,Baseball,False,1,0,7,4,7,1
202064,CHN,Baseball,False,1,0,7,4,7,1


In [176]:
Baseball_df['Predictions'] = y_pred.tolist()
Baseball_final = Baseball_df.drop_duplicates(subset=['NOC'], keep='last')
Baseball_final = Baseball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Baseball_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1263594060.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Baseball_final = Baseball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
186125,ESP,Baseball,0.062307
187383,ZAF,Baseball,0.007310
188471,ITA,Baseball,0.005539
188746,PRI,Baseball,0.000396
188969,CUB,Baseball,0.999967
190122,DOM,Baseball,0.000396
193818,CAN,Baseball,0.045414
195652,AUS,Baseball,0.603954
197203,NLD,Baseball,0.025387
199761,KOR,Baseball,0.999055


In [177]:
M = pd.merge(S_Swimming_final, Baseball_final, how='outer', on=['NOC', 'NOC'])
M.rename(columns = {'Predictions_x':'Sychronized Swimming', 'Predictions_y':'Baseball'}, inplace = True)
M = M.drop(["Sport_x","Sport_y"], 1)
M

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/184479326.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  M = M.drop(["Sport_x","Sport_y"], 1)


,NOC,Sychronized Swimming,Baseball
0,COL,0.020002,NaN
1,SWE,0.031102,NaN
2,SVK,0.020002,NaN
3,FIN,0.031102,NaN
4,HUN,0.017296,NaN
5,CUB,0.020002,0.999967
6,VEN,0.012728,NaN
7,NZL,0.012728,NaN
8,ARG,0.073294,NaN
9,DEU,0.086904,NaN


Rhythmic Gymnastics

In [178]:
R_Gymnastics_df = data_df[data_df['Sport'] == "Rhythmic Gymnastics"]
R_Gymnastics_df = R_Gymnastics_df.dropna()

In [179]:
R_Gymnastics_dummies = pd.get_dummies(R_Gymnastics_df)
R_Gymnastics_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
527,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
759,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
935,False,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [180]:
x_train = R_Gymnastics_dummies.drop(columns=['Medal'])
y_train = R_Gymnastics_dummies['Medal']

x_test = R_Gymnastics_dummies.drop(columns=['Medal'])
y_test = R_Gymnastics_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8769716088328076

In [181]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,R_Gymnastics_df)

array([0.4298816 , 0.07215702, 0.78556536, 0.47914163, 0.47914163,
       0.47914163, 0.45257012, 0.023882  , 0.00400326, 0.75057961,
       0.34272525, 0.98023693, 0.07215702, 0.71021775, 0.07215702,
       0.45257012, 0.75057961, 0.28454195, 0.28454195, 0.03239036,
       0.61050046, 0.07215702, 0.03239036, 0.69286968, 0.04204346,
       0.75057961, 0.07215702, 0.61050046, 0.07215702, 0.25740155,
       0.03443816, 0.98744325, 0.96509578, 0.75057961, 0.98744325,
       0.98744325, 0.13854564, 0.03239036, 0.03390216, 0.03239036,
       0.07215702, 0.55763103, 0.03481211, 0.0073689 , 0.36851735,
       0.38420905, 0.07215702, 0.36851735, 0.03390216, 0.98023693,
       0.07215702, 0.03239036, 0.04204346, 0.07181265, 0.54108   ,
       0.55763103, 0.55763103, 0.98023693, 0.98023693, 0.07215702,
       0.61050046, 0.54108   , 0.96509578, 0.07215702, 0.07215702,
       0.0524677 , 0.03239036, 0.75057961, 0.75057961, 0.04052246,
       0.61050046, 0.80972914, 0.07215702, 0.61050046, 0.03239

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
527,ESP,Rhythmic Gymnastics,False,0,1,0,2,8,0
759,GEO,Rhythmic Gymnastics,False,0,0,0,4,7,0
935,ESP,Rhythmic Gymnastics,False,1,0,0,2,8,0
961,ESP,Rhythmic Gymnastics,False,0,1,0,1,8,0
1783,ESP,Rhythmic Gymnastics,False,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202970,RUS,Rhythmic Gymnastics,True,0,0,0,2,6,0
203071,USA,Rhythmic Gymnastics,False,1,0,1,2,7,7
203465,NOR,Rhythmic Gymnastics,False,0,0,0,2,8,0
203608,UKR,Rhythmic Gymnastics,False,0,0,0,2,7,0


In [182]:
R_Gymnastics_df['Predictions'] = y_pred.tolist()
R_Gymnastics_final = R_Gymnastics_df.drop_duplicates(subset=['NOC'], keep='last')
R_Gymnastics_final = R_Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
R_Gymnastics_final

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2322458286.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  R_Gymnastics_final = R_Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
14839,SWE,Rhythmic Gymnastics,0.138546
53785,DNK,Rhythmic Gymnastics,0.046528
82268,PRT,Rhythmic Gymnastics,0.072157
88051,EST,Rhythmic Gymnastics,0.076764
90843,LTU,Rhythmic Gymnastics,0.072157
121737,CPV,Rhythmic Gymnastics,0.072157
145369,AUS,Rhythmic Gymnastics,0.042397
157925,ZAF,Rhythmic Gymnastics,0.109766
160902,BRA,Rhythmic Gymnastics,0.044298
162979,FRA,Rhythmic Gymnastics,0.059635


Taekwondo

In [183]:
Taekwondo_df = data_df[data_df['Sport'] == "Taekwondo"]
Taekwondo_df = Taekwondo_df.dropna()

In [184]:
Taekwondo_dummies = pd.get_dummies(Taekwondo_df)
Taekwondo_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
92,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
101,True,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
213,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [185]:
x_train = Taekwondo_dummies.drop(columns=['Medal'])
y_train = Taekwondo_dummies['Medal']

x_test = Taekwondo_dummies.drop(columns=['Medal'])
y_test = Taekwondo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.7979966611018364

In [186]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Taekwondo_df)

array([0.24843322, 0.51421195, 0.24843322, 0.76555365, 0.15769982,
       0.15769982, 0.40696182, 0.40696182, 0.1647951 , 0.41249512,
       0.24843322, 0.57544734, 0.24843322, 0.23299512, 0.40696182,
       0.40696182, 0.15769982, 0.25540956, 0.15769982, 0.55086805,
       0.51919866, 0.60968579, 0.10354845, 0.23299512, 0.22849847,
       0.24843322, 0.13082621, 0.1647951 , 0.60968579, 0.25540956,
       0.0847481 , 0.51689638, 0.58378004, 0.1647951 , 0.29326519,
       0.6888633 , 0.74152726, 0.40696182, 0.27874656, 0.41110975,
       0.63574567, 0.51823662, 0.23299512, 0.62474155, 0.0847481 ,
       0.15769982, 0.28106996, 0.56620771, 0.56620771, 0.52799216,
       0.67473598, 0.76040854, 0.18985999, 0.51421195, 0.20002041,
       0.11228419, 0.11228419, 0.23299512, 0.23299512, 0.23299512,
       0.23299512, 0.52026637, 0.25930275, 0.11228419, 0.6526662 ,
       0.23299512, 0.51508553, 0.52595716, 0.68970168, 0.38067343,
       0.38067343, 0.18680796, 0.18680796, 0.11228419, 0.11228

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
92,EGY,Taekwondo,False,0,0,0,5,7,0
101,AZE,Taekwondo,True,0,0,0,4,7,0
213,EGY,Taekwondo,False,0,0,0,5,7,0
230,USA,Taekwondo,True,0,1,1,1,8,6
475,KGZ,Taekwondo,False,0,0,0,6,7,0
...,...,...,...,...,...,...,...,...,...
203239,AZE,Taekwondo,False,0,0,0,4,7,0
203240,AZE,Taekwondo,False,0,0,0,4,7,0
203565,MAR,Taekwondo,False,0,0,0,4,7,0
203566,MAR,Taekwondo,False,0,0,0,4,7,0


In [187]:
Taekwondo_df['Predictions'] = y_pred.tolist()
Taekwondo_final = Taekwondo_df.drop_duplicates(subset=['NOC'], keep='last')
Taekwondo_final = Taekwondo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/4211559542.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Taekwondo_final = Taekwondo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [188]:
N = pd.merge(R_Gymnastics_final, Taekwondo_final, how='outer', on=['NOC', 'NOC'])
N.rename(columns = {'Predictions_x':'Rhythmic Gymnastics', 'Predictions_y':'Taekwondo'}, inplace = True)
N = N.drop(["Sport_x","Sport_y"], 1)
N

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/1912829512.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  N = N.drop(["Sport_x","Sport_y"], 1)


,NOC,Rhythmic Gymnastics,Taekwondo
0,SWE,0.138546,0.175486
1,DNK,0.046528,0.255410
2,PRT,0.072157,0.178183
3,EST,0.076764,NaN
4,LTU,0.072157,NaN
...,...,...,...
104,THA,NaN,0.667129
105,ARM,NaN,0.232995
106,GTM,NaN,0.293265
107,MAR,NaN,0.112284


Beach Volleyball

In [189]:
B_Volleyball_df = data_df[data_df['Sport'] == "Beach Volleyball"]
B_Volleyball_df = B_Volleyball_df.dropna()

In [190]:
B_Volleyball_dummies = pd.get_dummies(B_Volleyball_df)
B_Volleyball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
816,False,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1912,False,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1913,True,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [191]:
x_train = B_Volleyball_dummies.drop(columns=['Medal'])
y_train = B_Volleyball_dummies['Medal']

x_test = B_Volleyball_dummies.drop(columns=['Medal'])
y_test = B_Volleyball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8714285714285714

In [192]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,B_Volleyball_df)

array([0.00298408, 0.48522136, 0.48522136, 0.07281862, 0.86545323,
       0.90233933, 0.07281862, 0.07281862, 0.40513116, 0.86545323,
       0.86545323, 0.027459  , 0.64646007, 0.07975216, 0.07975216,
       0.07975216, 0.12146575, 0.27665455, 0.027459  , 0.25739947,
       0.30035084, 0.30035084, 0.0406261 , 0.08352734, 0.08352734,
       0.90233933, 0.45934076, 0.06914769, 0.07281862, 0.07281862,
       0.06914769, 0.64646007, 0.90233933, 0.86545323, 0.86545323,
       0.027459  , 0.07975216, 0.09158174, 0.07281862, 0.86545323,
       0.04464197, 0.04464197, 0.0302143 , 0.0302143 , 0.07094512,
       0.30820262, 0.25739947, 0.7835014 , 0.7835014 , 0.48522136,
       0.21356752, 0.48522136, 0.41911841, 0.41911841, 0.44326166,
       0.76749177, 0.0406261 , 0.0406261 , 0.0406261 , 0.5478188 ,
       0.5478188 , 0.25739947, 0.36811902, 0.36887596, 0.18158747,
       0.027459  , 0.48522136, 0.48522136, 0.07281862, 0.20872721,
       0.40513116, 0.87247834, 0.07281862, 0.07281862, 0.07281

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
816,AGO,Beach Volleyball,False,0,0,0,5,5,0
1912,DEU,Beach Volleyball,False,0,1,0,2,8,1
1913,DEU,Beach Volleyball,True,0,1,0,2,8,1
3764,CRI,Beach Volleyball,False,0,0,0,2,8,0
3948,BRA,Beach Volleyball,True,0,0,1,1,7,0
...,...,...,...,...,...,...,...,...,...
202179,CHN,Beach Volleyball,True,1,0,7,4,7,1
202180,CHN,Beach Volleyball,False,0,0,7,4,7,2
203411,ARG,Beach Volleyball,False,0,0,0,0,7,0
203768,CHE,Beach Volleyball,False,0,3,0,2,8,0


In [193]:
B_Volleyball_df['Predictions'] = y_pred.tolist()
B_Volleyball_final = B_Volleyball_df.drop_duplicates(subset=['NOC'], keep='last')
B_Volleyball_final = B_Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3648103712.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  B_Volleyball_final = B_Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Triathlon

In [194]:
Triathlon_df = data_df[data_df['Sport'] == "Triathlon"]
Triathlon_df = Triathlon_df.dropna()

In [195]:
Triathlon_dummies = pd.get_dummies(Triathlon_df)
Triathlon_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BRA,NOC_BRB,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
831,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
901,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
902,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [196]:
x_train = Triathlon_dummies.drop(columns=['Medal'])
y_train = Triathlon_dummies['Medal']

x_test = Triathlon_dummies.drop(columns=['Medal'])
y_test = Triathlon_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.941972920696325

In [197]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Triathlon_df)

array([3.76759683e-02, 3.76759683e-02, 3.76759683e-02, 4.68804276e-04,
       4.66100333e-01, 5.30373215e-01, 3.76759683e-02, 3.76759683e-02,
       5.27926015e-01, 4.66100333e-01, 8.17104579e-03, 1.40698264e-01,
       1.40698264e-01, 7.57376197e-01, 4.68545390e-01, 8.85095325e-02,
       7.76376794e-01, 7.54847780e-01, 1.88094392e-01, 3.76759683e-02,
       9.90541778e-03, 1.74746456e-01, 9.42069912e-02, 8.88302189e-02,
       5.42301218e-01, 1.44158116e-01, 2.40347371e-03, 1.87726319e-03,
       8.16423672e-01, 5.26925514e-01, 3.76808216e-01, 8.57422022e-01,
       2.07966815e-01, 4.48672234e-01, 4.48672234e-01, 4.65491140e-02,
       5.26925514e-01, 9.90541778e-03, 4.65491140e-02, 2.40347371e-03,
       4.38802501e-01, 4.68545390e-01, 5.30373215e-01, 1.07302113e-01,
       1.07302113e-01, 8.83061711e-03, 3.33649977e-02, 5.27926015e-01,
       4.66100333e-01, 5.80357344e-01, 8.67140407e-02, 2.36877816e-02,
       2.28850603e-01, 6.91062705e-02, 1.88094392e-01, 8.57422022e-01,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
831,RUS,Triathlon,False,0,0,0,2,7,0
901,RUS,Triathlon,False,0,0,0,2,7,0
902,RUS,Triathlon,False,0,0,0,2,7,0
1078,JPN,Triathlon,False,0,2,0,0,8,1
1685,AUT,Triathlon,False,0,2,0,3,8,0
...,...,...,...,...,...,...,...,...,...
201588,BEL,Triathlon,False,0,2,0,0,8,0
201620,USA,Triathlon,False,0,1,1,1,8,6
201646,CZE,Triathlon,False,0,0,0,2,7,0
202244,CHN,Triathlon,False,1,0,7,4,7,1


In [198]:
Triathlon_df['Predictions'] = y_pred.tolist()
Triathlon_final = Triathlon_df.drop_duplicates(subset=['NOC'], keep='last')
Triathlon_final = Triathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/2330985694.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Triathlon_final = Triathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [199]:
O = pd.merge(B_Volleyball_final, Triathlon_final, how='outer', on=['NOC', 'NOC'])
O.rename(columns = {'Predictions_x':'Beach Volleyball', 'Predictions_y':'Triathlon'}, inplace = True)
O = O.drop(["Sport_x","Sport_y"], 1)
O

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3167638783.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  O = O.drop(["Sport_x","Sport_y"], 1)


,NOC,Beach Volleyball,Triathlon
0,TUN,0.027459,NaN
1,CRI,0.072819,0.023688
2,AGO,0.002984,NaN
3,CHL,0.083527,0.033365
4,NZL,0.040626,0.069106
5,EGY,0.002984,NaN
6,GBR,0.062899,0.857422
7,PRI,0.083527,NaN
8,SWE,0.044642,0.788082
9,POL,0.027459,0.023688


Softball

In [200]:
Softball_df = data_df[data_df['Sport'] == "Softball"]
Softball_df = Softball_df.dropna()

In [201]:
Softball_dummies = pd.get_dummies(Softball_df)
Softball_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_CAN,NOC_CHN,NOC_CUB,NOC_GRC,NOC_ITA,NOC_JPN,NOC_NLD,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
205,True,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2088,False,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
3904,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [202]:
x_train = Softball_dummies.drop(columns=['Medal'])
y_train = Softball_dummies['Medal']

x_test = Softball_dummies.drop(columns=['Medal'])
y_test = Softball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9686192468619247

In [203]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Softball_df)

array([9.98711055e-01, 1.50561102e-04, 2.30337889e-02, 9.92797885e-01,
       9.92797885e-01, 9.71746722e-01, 1.50561102e-04, 9.99678087e-01,
       5.94289061e-01, 5.53152039e-03, 1.25473622e-01, 9.57381529e-01,
       1.96310668e-02, 2.30337889e-02, 1.50561102e-04, 3.69645737e-02,
       1.50561102e-04, 1.50561102e-04, 2.30337889e-02, 1.96310668e-02,
       1.96310668e-02, 1.96310668e-02, 3.69645737e-02, 3.69645737e-02,
       9.48609528e-03, 1.84356645e-06, 2.25557764e-03, 2.25557764e-03,
       9.52001945e-01, 9.99678087e-01, 9.99678087e-01, 9.98711055e-01,
       8.11475442e-02, 2.30337889e-02, 1.96310668e-02, 9.71746722e-01,
       9.52001945e-01, 3.69645737e-02, 9.48609528e-03, 9.92797885e-01,
       9.92797885e-01, 9.99678087e-01, 1.50561102e-04, 1.96310668e-02,
       9.99678087e-01, 9.99678087e-01, 9.98711055e-01, 9.92797885e-01,
       3.69645737e-02, 1.50561102e-04, 1.96310668e-02, 2.30337889e-02,
       1.50561102e-04, 1.50561102e-04, 5.94289061e-01, 2.92579696e-02,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
205,USA,Softball,True,0,2,1,1,8,5
2088,ITA,Softball,False,0,0,0,3,8,0
3904,VEN,Softball,False,0,0,0,1,7,0
4121,AUS,Softball,True,1,1,0,1,8,0
4122,AUS,Softball,True,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202090,CHN,Softball,False,0,0,8,5,7,1
202240,CHN,Softball,False,0,0,8,5,7,0
202668,CHN,Softball,False,0,0,8,5,7,0
202676,CHN,Softball,False,0,0,8,5,7,1


In [204]:
Softball_df['Predictions'] = y_pred.tolist()
Softball_final = Softball_df.drop_duplicates(subset=['NOC'], keep='last')
Softball_final = Softball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3040282946.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Softball_final = Softball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Rugby Sevens

In [205]:
Rugby_df = data_df[data_df['Sport'] == "Rugby Sevens"]
Rugby_df = Rugby_df.dropna()

In [206]:
Rugby_dummies = pd.get_dummies(Rugby_df)
Rugby_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_BRA,NOC_CAN,NOC_COL,NOC_ESP,NOC_FJI,NOC_FRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
605,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
950,False,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
951,False,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [207]:
x_train = Rugby_dummies.drop(columns=['Medal'])
y_train = Rugby_dummies['Medal']

x_test = Rugby_dummies.drop(columns=['Medal'])
y_test = Rugby_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8361204013377926

In [208]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Rugby_df)

array([0.06250894, 0.11392208, 0.11392208, 0.06250894, 0.96192436,
       0.96192436, 0.06020067, 0.00715048, 0.72146844, 0.11577053,
       0.06250894, 0.06020067, 0.06250894, 0.06250894, 0.00715048,
       0.06250894, 0.05304505, 0.70688934, 0.00715048, 0.05304505,
       0.05304505, 0.06020067, 0.70688934, 0.05304505, 0.95875139,
       0.72146844, 0.00715048, 0.11392208, 0.72146844, 0.06020067,
       0.06020067, 0.06020067, 0.70688934, 0.70688934, 0.72146844,
       0.96192436, 0.11577053, 0.72146844, 0.00715048, 0.06020067,
       0.05304505, 0.06020067, 0.06020067, 0.70688934, 0.00715048,
       0.06250894, 0.70688934, 0.70688934, 0.00715048, 0.06020067,
       0.00715048, 0.72146844, 0.7231727 , 0.70688934, 0.70688934,
       0.7231727 , 0.06020067, 0.70688934, 0.95875139, 0.7231727 ,
       0.72146844, 0.96192436, 0.06020067, 0.11392208, 0.70688934,
       0.7231727 , 0.05304505, 0.00715048, 0.00715048, 0.05304505,
       0.05304505, 0.06020067, 0.06020067, 0.11577053, 0.11577

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
605,KEN,Rugby Sevens,False,0,0,0,6,7,0
950,COL,Rugby Sevens,False,0,0,0,2,7,0
951,COL,Rugby Sevens,False,0,0,0,2,7,0
1311,KEN,Rugby Sevens,False,0,0,0,6,7,0
1555,ZAF,Rugby Sevens,True,0,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
197026,NZL,Rugby Sevens,True,0,1,0,1,8,0
197656,USA,Rugby Sevens,False,0,2,1,1,8,5
198269,JPN,Rugby Sevens,False,0,1,0,0,8,1
198376,JPN,Rugby Sevens,False,0,1,0,0,8,1


In [209]:
Rugby_df['Predictions'] = y_pred.tolist()
Rugby_final = Rugby_df.drop_duplicates(subset=['NOC'], keep='last')
Rugby_final = Rugby_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3353199450.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Rugby_final = Rugby_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [210]:
P = pd.merge(Softball_final, Rugby_final, how='outer', on=['NOC', 'NOC'])
P.rename(columns = {'Predictions_x':'Softball', 'Predictions_y':'Rugby Sevens'}, inplace = True)
P = P.drop(["Sport_x","Sport_y"], 1)
P

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3810029682.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  P = P.drop(["Sport_x","Sport_y"], 1)


,NOC,Softball,Rugby Sevens
0,GRC,0.019631,NaN
1,VEN,0.023034,NaN
2,PRI,0.000151,NaN
3,ITA,0.019631,NaN
4,NLD,0.081148,NaN
5,NZL,0.023034,0.706889
6,USA,0.999678,0.053045
7,AUS,0.971747,0.706889
8,JPN,0.957382,0.060201
9,CAN,0.009486,0.958751


Trampolining

In [211]:
Trampolining_df = data_df[data_df['Sport'] == "Trampolining"]
Trampolining_df = Trampolining_df.dropna()

In [212]:
Trampolining_dummies = pd.get_dummies(Trampolining_df)
Trampolining_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_BLR,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,NOC_CZE,NOC_DEU,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
803,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1099,False,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
2029,False,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [213]:
x_train = Trampolining_dummies.drop(columns=['Medal'])
y_train = Trampolining_dummies['Medal']

x_test = Trampolining_dummies.drop(columns=['Medal'])
y_test = Trampolining_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8552631578947368

In [214]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Trampolining_df)

array([0.22932361, 0.48264506, 0.11771699, 0.43035389, 0.42786484,
       0.24940217, 0.11771699, 0.43035389, 0.24940217, 0.69901134,
       0.69901134, 0.8426958 , 0.24940217, 0.17393471, 0.17393471,
       0.4677557 , 0.69694772, 0.32374852, 0.8426958 , 0.8426958 ,
       0.69901134, 0.69901134, 0.29490856, 0.48264506, 0.48264506,
       0.28796712, 0.28796712, 0.93994354, 0.93608182, 0.86561172,
       0.11771699, 0.26204482, 0.24940217, 0.11771699, 0.32374852,
       0.30890864, 0.17393471, 0.15402369, 0.22932361, 0.22932361,
       0.86561172, 0.29490856, 0.15348926, 0.11771699, 0.32111815,
       0.32111815, 0.32111815, 0.32374852, 0.32374852, 0.93994354,
       0.93608182, 0.86561172, 0.15402369, 0.15402369, 0.80284836,
       0.93994354, 0.93608182, 0.07315438, 0.15402369, 0.12590809,
       0.12590809, 0.12590809, 0.24940217, 0.69694772, 0.69694772,
       0.32374852, 0.32374852, 0.32374852, 0.32374852, 0.43035389,
       0.43035389, 0.43035389, 0.43035389, 0.43035389, 0.32111

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
803,PRT,Trampolining,False,0,0,0,3,8,0
1099,DEU,Trampolining,False,0,1,0,2,8,1
2029,USA,Trampolining,False,0,2,1,1,8,5
3963,KAZ,Trampolining,False,0,0,0,4,7,0
5722,BRA,Trampolining,False,1,0,1,1,7,0
...,...,...,...,...,...,...,...,...,...
192188,AUS,Trampolining,True,1,1,0,1,8,0
195202,CHE,Trampolining,False,0,1,0,2,8,0
195374,AUS,Trampolining,False,0,2,0,1,8,0
197277,GBR,Trampolining,False,0,2,0,1,8,1


In [215]:
Trampolining_df['Predictions'] = y_pred.tolist()
Trampolining_final = Trampolining_df.drop_duplicates(subset=['NOC'], keep='last')
Trampolining_final = Trampolining_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/458389408.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Trampolining_final = Trampolining_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Golf

In [216]:
Golf_df = data_df[data_df['Sport'] == "Golf"]
Golf_df = Golf_df.dropna()

In [217]:
Golf_dummies = pd.get_dummies(Golf_df)
Golf_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGD,NOC_BRA,NOC_CAN,NOC_CHE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
1755,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
5152,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
6701,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [218]:
x_train = Golf_dummies.drop(columns=['Medal'])
y_train = Golf_dummies['Medal']

x_test = Golf_dummies.drop(columns=['Medal'])
y_test = Golf_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9491525423728814

In [219]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Golf_df)

array([0.07931216, 0.70848206, 0.0338479 , 0.14224461, 0.07931216,
       0.07931216, 0.19734278, 0.19734278, 0.19734278, 0.1744123 ,
       0.14224461, 0.0338479 , 0.70848206, 0.19734278, 0.07931216,
       0.32125143, 0.19734278, 0.65331279, 0.74053366, 0.81137233,
       0.07931216, 0.1744123 , 0.19734278, 0.07931216, 0.07931216,
       0.0338479 , 0.0338479 , 0.07931216, 0.0338479 , 0.07931216,
       0.19734278, 0.07931216, 0.19734278, 0.77392438, 0.19734278,
       0.1744123 , 0.19734278, 0.07931216, 0.0338479 , 0.0338479 ,
       0.19734278, 0.1744123 , 0.1744123 , 0.70848206, 0.19734278,
       0.81137233, 0.19734278, 0.0338479 , 0.74053366, 0.65331279,
       0.14224461, 0.19734278, 0.81137233, 0.07931216, 0.07931216,
       0.74053366, 0.65331279, 0.07931216, 0.65331279, 0.65331279,
       0.77392438, 0.77392438, 0.19734278, 0.19734278, 0.32125143,
       0.65331279, 0.07931216, 0.19734278, 0.07931216, 0.07931216,
       0.1744123 , 0.77392438, 0.07931216, 0.07931216, 0.19734

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
1755,CHL,Golf,False,0,0,0,0,8,0
5152,KOR,Golf,False,0,1,0,1,8,0
6701,THA,Golf,False,0,0,0,4,7,0
7986,IND,Golf,False,0,0,7,6,7,0
13296,ISR,Golf,False,0,1,0,0,8,0
...,...,...,...,...,...,...,...,...,...
195547,GBR,Golf,False,0,1,0,1,8,0
196704,AUT,Golf,False,0,2,0,3,8,0
197358,CHN,Golf,False,0,0,7,4,8,3
198605,KOR,Golf,False,0,1,0,1,8,0


In [220]:
Golf_df['Predictions'] = y_pred.tolist()
Golf_final = Golf_df.drop_duplicates(subset=['NOC'], keep='last')
Golf_final = Golf_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/3088466276.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Golf_final = Golf_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [221]:
Q = pd.merge(Trampolining_final, Golf_final, how='outer', on=['NOC', 'NOC'])
Q.rename(columns = {'Predictions_x':'Trampoling', 'Predictions_y':'Golf'}, inplace = True)
Q = Q.drop(["Sport_x","Sport_y"], 1)
Q

C:\Users\14043\AppData\Local\Temp/ipykernel_6344/762997901.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Q = Q.drop(["Sport_x","Sport_y"], 1)


,NOC,Trampoling,Golf
0,KAZ,0.430354,NaN
1,BRA,0.427865,0.321251
2,DZA,0.430354,NaN
3,ITA,0.173935,0.079312
4,DNK,0.125908,0.197343
5,UZB,0.430354,NaN
6,GEO,0.321118,NaN
7,CHN,0.802848,0.653313
8,GRC,0.173935,NaN
9,FRA,0.249402,0.197343


In [222]:
First = pd.merge(A, B, how='outer', on=['NOC', 'NOC'])
First

,NOC,Athletics,Swimming,Gymnastics,Cycling
0,OMN,0.056705,0.035615,NaN,0.082341
1,PLW,0.142449,0.035615,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN
4,BDI,0.394084,0.007110,NaN,NaN
...,...,...,...,...,...
182,DEU,0.464096,0.635457,0.302231,0.766623
183,NLD,0.186879,0.642650,0.258108,0.579864
184,USA,0.858038,0.964735,0.631419,0.008194
185,KAZ,0.268187,0.073423,0.017182,0.389327


In [223]:
Second = pd.merge(C, D, how='outer', on=['NOC', 'NOC'])
Second

,NOC,Rowing,Shooting,Fencing,Canoeing
0,KWT,0.191545,0.534135,0.031137,NaN
1,TGO,0.008688,NaN,NaN,0.689694
2,HND,0.091263,0.190863,0.043281,NaN
3,TTO,0.186649,0.145883,NaN,NaN
4,CMR,0.103479,NaN,NaN,NaN
...,...,...,...,...,...
136,ATG,NaN,NaN,NaN,0.030552
137,SYC,NaN,NaN,NaN,0.106971
138,MOZ,NaN,NaN,NaN,0.023625
139,GUM,NaN,NaN,NaN,0.310505


In [224]:
Third = pd.merge(E, F, how='outer', on=['NOC', 'NOC'])
Third

,NOC,Wrestling,Football,Boxing,Equestrianism
0,YEM,0.024007,NaN,NaN,NaN
1,CHL,0.196642,0.892595,0.094681,0.008917
2,BHR,0.298731,NaN,NaN,NaN
3,LBN,0.165726,NaN,0.274801,NaN
4,CIV,0.050638,0.000010,0.084473,NaN
...,...,...,...,...,...
158,ETH,NaN,NaN,0.133539,NaN
159,TON,NaN,NaN,0.257780,NaN
160,JAM,NaN,NaN,0.061504,0.112333
161,PNG,NaN,NaN,0.096752,NaN


In [225]:
Fourth = pd.merge(G, H, how='outer', on=['NOC', 'NOC'])
Fourth

,NOC,Judo,Basketball,Handball,Volleyball
0,GUM,0.133953,NaN,NaN,NaN
1,TGO,0.068391,NaN,NaN,NaN
2,NIC,0.180088,NaN,NaN,NaN
3,TTO,0.180088,NaN,NaN,NaN
4,BEN,0.068391,NaN,NaN,NaN
...,...,...,...,...,...
152,HUN,0.430879,NaN,0.574324,NaN
153,CHL,0.174663,NaN,NaN,NaN
154,CAN,0.194537,0.002087,0.014531,0.018648
155,GEO,0.579908,NaN,NaN,NaN


In [226]:
Fifth = pd.merge(I, J, how='outer', on=['NOC', 'NOC'])
Fifth

,NOC,Weightlifting,Water Polo,Archery,Diving
0,SAU,0.199777,NaN,0.011222,NaN
1,GHA,0.055787,NaN,NaN,NaN
2,ZWE,0.018350,NaN,0.026751,0.025319
3,SLE,0.060117,NaN,NaN,NaN
4,LBN,0.199777,NaN,NaN,NaN
...,...,...,...,...,...
133,LAO,NaN,NaN,0.026751,NaN
134,CAF,NaN,NaN,0.050457,NaN
135,HKG,NaN,NaN,0.156893,0.033800
136,BTN,NaN,NaN,0.050457,NaN


In [227]:
Sixth = pd.merge(K, L, how='outer', on=['NOC', 'NOC'])
Sixth

,NOC,Tennis,Table Tennis,Badminton,Modern Pentathlon
0,MDA,0.167711,NaN,NaN,NaN
1,SLV,0.042138,NaN,NaN,NaN
2,LTU,0.042138,0.006648,0.001491,0.790229
3,TUR,0.117731,0.006648,0.033922,0.111346
4,BOL,0.030837,NaN,NaN,NaN
...,...,...,...,...,...
116,LKA,NaN,NaN,0.063675,NaN
117,GTM,NaN,NaN,0.042557,0.070235
118,BRN,NaN,NaN,0.016015,NaN
119,BHR,NaN,NaN,NaN,0.153452


In [228]:
Seventh = pd.merge(M, N, how='outer', on=['NOC', 'NOC'])
Seventh

,NOC,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo
0,COL,0.020002,NaN,NaN,0.391344
1,SWE,0.031102,NaN,0.138546,0.175486
2,SVK,0.020002,NaN,NaN,NaN
3,FIN,0.031102,NaN,0.075022,0.255410
4,HUN,0.017296,NaN,0.072157,0.232995
...,...,...,...,...,...
105,THA,NaN,NaN,NaN,0.667129
106,ARM,NaN,NaN,NaN,0.232995
107,GTM,NaN,NaN,NaN,0.293265
108,MAR,NaN,NaN,NaN,0.112284


In [229]:
Eighth = pd.merge(O, P, how='outer', on=['NOC', 'NOC'])
Eighth 

,NOC,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,TUN,0.027459,NaN,NaN,NaN
1,CRI,0.072819,0.023688,NaN,NaN
2,AGO,0.002984,NaN,NaN,NaN
3,CHL,0.083527,0.033365,NaN,NaN
4,NZL,0.040626,0.069106,0.023034,0.706889
5,EGY,0.002984,NaN,NaN,NaN
6,GBR,0.062899,0.857422,NaN,0.721468
7,PRI,0.083527,NaN,0.000151,NaN
8,SWE,0.044642,0.788082,NaN,NaN
9,POL,0.027459,0.023688,NaN,NaN


In [230]:
A = pd.merge(First, Second, how='outer', on=['NOC', 'NOC'])
A 

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing
0,OMN,0.056705,0.035615,NaN,0.082341,NaN,0.086523,NaN,NaN
1,PLW,0.142449,0.035615,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN,0.008688,0.007465,NaN,0.160368
4,BDI,0.394084,0.007110,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
183,NLD,0.186879,0.642650,0.258108,0.579864,0.589524,0.192873,0.256287,0.336494
184,USA,0.858038,0.964735,0.631419,0.008194,0.404300,0.775114,0.214376,0.330400
185,KAZ,0.268187,0.073423,0.017182,0.389327,0.091263,0.412304,0.008773,0.027575
186,RUS,0.735707,0.499311,0.786959,0.543147,0.382251,0.875085,0.838269,0.586214


In [231]:
B = pd.merge(Third, Fourth, how = 'outer', on=['NOC','NOC'])
B

,NOC,Wrestling,Football,Boxing,Equestrianism,Judo,Basketball,Handball,Volleyball
0,YEM,0.024007,NaN,NaN,NaN,0.002548,NaN,NaN,NaN
1,CHL,0.196642,0.892595,0.094681,0.008917,0.174663,NaN,NaN,NaN
2,BHR,0.298731,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LBN,0.165726,NaN,0.274801,NaN,0.172859,NaN,NaN,NaN
4,CIV,0.050638,0.000010,0.084473,NaN,0.035575,NaN,0.009623,NaN
...,...,...,...,...,...,...,...,...,...
169,ISL,NaN,NaN,NaN,NaN,0.236407,NaN,0.478190,NaN
170,RWA,NaN,NaN,NaN,NaN,0.048660,NaN,NaN,NaN
171,ABW,NaN,NaN,NaN,NaN,0.180088,NaN,NaN,NaN
172,MMR,NaN,NaN,NaN,NaN,0.079566,NaN,NaN,NaN


In [232]:
C = pd.merge(Fifth, Sixth, how = 'outer', on=['NOC','NOC'])
C

,NOC,Weightlifting,Water Polo,Archery,Diving,Tennis,Table Tennis,Badminton,Modern Pentathlon
0,SAU,0.199777,NaN,0.011222,NaN,NaN,0.006648,NaN,NaN
1,GHA,0.055787,NaN,NaN,NaN,NaN,0.007065,NaN,NaN
2,ZWE,0.018350,NaN,0.026751,0.025319,0.029888,NaN,NaN,NaN
3,SLE,0.060117,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LBN,0.199777,NaN,NaN,NaN,NaN,0.002929,NaN,NaN
...,...,...,...,...,...,...,...,...,...
148,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,0.055207,NaN
149,MDV,NaN,NaN,NaN,NaN,NaN,NaN,0.055207,NaN
150,SUR,NaN,NaN,NaN,NaN,NaN,NaN,0.001491,NaN
151,BRN,NaN,NaN,NaN,NaN,NaN,NaN,0.016015,NaN


In [233]:
D = pd.merge(Seventh, Eighth, how = 'outer', on=['NOC','NOC'])
D

,NOC,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,COL,0.020002,NaN,NaN,0.391344,NaN,0.037676,NaN,0.113922
1,SWE,0.031102,NaN,0.138546,0.175486,0.044642,0.788082,NaN,NaN
2,SVK,0.020002,NaN,NaN,NaN,NaN,0.009905,NaN,NaN
3,FIN,0.031102,NaN,0.075022,0.255410,NaN,NaN,NaN,NaN
4,HUN,0.017296,NaN,0.072157,0.232995,NaN,0.037676,NaN,NaN
...,...,...,...,...,...,...,...,...,...
113,ZWE,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN
114,LUX,NaN,NaN,NaN,NaN,NaN,0.468545,NaN,NaN
115,IRL,NaN,NaN,NaN,NaN,NaN,0.376766,NaN,NaN
116,BRB,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN


In [234]:
First = pd.merge(A, B, how = 'outer', on=['NOC','NOC'])
First

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,Football,Boxing,Equestrianism,Judo,Basketball,Handball,Volleyball
0,OMN,0.056705,0.035615,NaN,0.082341,NaN,0.086523,NaN,NaN,NaN,NaN,0.270816,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142449,0.035615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN,0.008688,0.007465,NaN,0.160368,NaN,NaN,0.063604,NaN,0.060364,0.012430,0.022963,NaN
4,BDI,0.394084,0.007110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186879,0.642650,0.258108,0.579864,0.589524,0.192873,0.256287,0.336494,0.334025,0.103051,0.551329,0.484679,0.643777,NaN,0.040047,0.426191
184,USA,0.858038,0.964735,0.631419,0.008194,0.404300,0.775114,0.214376,0.330400,0.767264,0.019816,0.793271,0.819402,0.538364,0.986470,0.000240,0.519982
185,KAZ,0.268187,0.073423,0.017182,0.389327,0.091263,0.412304,0.008773,0.027575,0.712686,NaN,0.800878,NaN,0.297577,NaN,0.027503,0.124527
186,RUS,0.735707,0.499311,0.786959,0.543147,0.382251,0.875085,0.838269,0.586214,0.924471,NaN,0.843003,0.104270,0.694117,0.880325,0.804634,0.738557


In [235]:
Second = pd.merge(C, D, how = 'outer', on=['NOC','NOC'])
Second

,NOC,Weightlifting,Water Polo,Archery,Diving,Tennis,Table Tennis,Badminton,Modern Pentathlon,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,SAU,0.199777,NaN,0.011222,NaN,NaN,0.006648,NaN,NaN,NaN,NaN,NaN,0.406962,NaN,NaN,NaN,NaN
1,GHA,0.055787,NaN,NaN,NaN,NaN,0.007065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZWE,0.018350,NaN,0.026751,0.025319,0.029888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN
3,SLE,0.060117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LBN,0.199777,NaN,NaN,NaN,NaN,0.002929,NaN,NaN,NaN,NaN,NaN,0.272006,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,GAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406962,NaN,NaN,NaN,NaN
160,KHM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.232995,NaN,NaN,NaN,NaN
161,GRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.157700,NaN,NaN,NaN,NaN
162,LSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.232995,NaN,NaN,NaN,NaN


In [236]:
A = pd.merge(First, Second, how = 'outer', on=['NOC','NOC'])
A

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,...,Badminton,Modern Pentathlon,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,OMN,0.056705,0.035615,NaN,0.082341,NaN,0.086523,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142449,0.035615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN,0.008688,0.007465,NaN,0.160368,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.002984,NaN,NaN,NaN
4,BDI,0.394084,0.007110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186879,0.642650,0.258108,0.579864,0.589524,0.192873,0.256287,0.336494,0.334025,...,0.226781,0.166414,0.048362,0.025387,NaN,0.263090,0.368876,0.088830,0.081148,NaN
184,USA,0.858038,0.964735,0.631419,0.008194,0.404300,0.775114,0.214376,0.330400,0.767264,...,0.038649,0.472360,0.772748,0.968680,0.034812,0.765554,0.767492,0.530373,0.999678,0.053045
185,KAZ,0.268187,0.073423,0.017182,0.389327,0.091263,0.412304,0.008773,0.027575,0.712686,...,NaN,0.490176,0.020002,NaN,0.072157,0.293265,NaN,0.140698,NaN,NaN
186,RUS,0.735707,0.499311,0.786959,0.543147,0.382251,0.875085,0.838269,0.586214,0.924471,...,0.274292,0.849722,0.456736,NaN,0.980237,0.566208,0.072819,0.037676,NaN,NaN


In [237]:
Final1 = pd.merge(A, Q, how = 'outer', on=['NOC','NOC'])
Final1

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,...,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens,Trampoling,Golf
0,OMN,0.056705,0.035615,NaN,0.082341,NaN,0.086523,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142449,0.035615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN,0.008688,0.007465,NaN,0.160368,NaN,...,NaN,NaN,NaN,NaN,0.002984,NaN,NaN,NaN,NaN,NaN
4,BDI,0.394084,0.007110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186879,0.642650,0.258108,0.579864,0.589524,0.192873,0.256287,0.336494,0.334025,...,0.048362,0.025387,NaN,0.263090,0.368876,0.088830,0.081148,NaN,0.249402,0.079312
184,USA,0.858038,0.964735,0.631419,0.008194,0.404300,0.775114,0.214376,0.330400,0.767264,...,0.772748,0.968680,0.034812,0.765554,0.767492,0.530373,0.999678,0.053045,0.117717,0.740534
185,KAZ,0.268187,0.073423,0.017182,0.389327,0.091263,0.412304,0.008773,0.027575,0.712686,...,0.020002,NaN,0.072157,0.293265,NaN,0.140698,NaN,NaN,0.430354,NaN
186,RUS,0.735707,0.499311,0.786959,0.543147,0.382251,0.875085,0.838269,0.586214,0.924471,...,0.456736,NaN,0.980237,0.566208,0.072819,0.037676,NaN,NaN,0.323749,0.033848


In [238]:
Final = pd.merge(Final1, Hockey_final, how = 'outer', on=['NOC','NOC'])
Final

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,...,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens,Trampoling,Golf,Hockey
0,OMN,0.056705,0.035615,NaN,0.082341,NaN,0.086523,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142449,0.035615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039488,0.006518,NaN,NaN,0.008688,0.007465,NaN,0.160368,NaN,...,NaN,NaN,NaN,0.002984,NaN,NaN,NaN,NaN,NaN,NaN
4,BDI,0.394084,0.007110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186879,0.642650,0.258108,0.579864,0.589524,0.192873,0.256287,0.336494,0.334025,...,0.025387,NaN,0.263090,0.368876,0.088830,0.081148,NaN,0.249402,0.079312,0.323228
184,USA,0.858038,0.964735,0.631419,0.008194,0.404300,0.775114,0.214376,0.330400,0.767264,...,0.968680,0.034812,0.765554,0.767492,0.530373,0.999678,0.053045,0.117717,0.740534,0.002332
185,KAZ,0.268187,0.073423,0.017182,0.389327,0.091263,0.412304,0.008773,0.027575,0.712686,...,NaN,0.072157,0.293265,NaN,0.140698,NaN,NaN,0.430354,NaN,NaN
186,RUS,0.735707,0.499311,0.786959,0.543147,0.382251,0.875085,0.838269,0.586214,0.924471,...,NaN,0.980237,0.566208,0.072819,0.037676,NaN,NaN,0.323749,0.033848,NaN


In [239]:
Final.to_csv(r'archive\countryPredictions.csv', index = False)